In [ ]:
from google.colab import drive
import os
import shutil
drive.mount('/content/gdrive')

In [ ]:
os.chdir('/content/gdrive/MyDrive/HIDE--main')

In [ ]:
import numpy as np

from utils import gaussjordan

from LinearCode import LinearCode


class QC_LDPC(LinearCode):
    """
    Quasi-Cyclic LDPC code representation (extends LinearCode)

    ...

    Methods
    -------
    from_params(n, p, w)
        Init QC-LDPC by length, circulant size and code weight

    _get_circulant_block(polynom)
        Get circulant (p, p) for given vector of size p

    """

    def __init__(self, G, H):
        super().__init__(G, H)

    @classmethod
    def from_params(cls, n, p, w):
        assert n % p == 0, "p must be delimeter of n"

        n0 = n // p
        assert w > 2*n0, "not enough code weight"

        fine = False

        while not fine:
            blocks = []
            inverse_block = None
            inverse_block_position = None

            vector = [1 for _ in range(w)] + [0 for _  in range(n - w)]
            vector = np.array(vector, dtype=int)
            np.random.shuffle(vector)

            for i in range(n0):
                circ = vector[i*p:(i+1)*p]

                if sum(circ) < 2:
                    inverse_block = None
                    break

                block = QC_LDPC._get_circulant_block(circ)
                blocks.append(block)

                A, P = gaussjordan(block, True)
                A = np.array(A, dtype=int)
                P = np.array(P, dtype=int)

                if (A == np.eye(p, dtype=int)).all():
                    inverse_block_position = i
                    inverse_block = P

            # continue only if inverse circulant found
            fine = True if inverse_block is not None else False

        # put inverse block on last position
        blocks[inverse_block_position], blocks[n0-1] = blocks[n0-1], blocks[inverse_block_position]
        H = np.concatenate(blocks, axis=1)

        for i in range(n0):
            blocks[i] = blocks[i] @ inverse_block % 2
            blocks[i] = blocks[i].T

        Ht = np.concatenate(blocks[:n0-1], axis=0)
        G = np.concatenate((np.eye(Ht.shape[0], dtype=int), Ht), axis=1)

        assert (G @ H.T % 2 == 0).all(), "G is not correspond to H"

        return cls(G, H)

    @staticmethod
    def _get_circulant_block(polynom):
        N = len(polynom)
        block = np.empty((N, N), dtype=int)

        for i in range(N):
            block[i] = np.roll(polynom, i)

        return block



# EXAMPLE USAGE:

n = 16
p = 8
w = 7

qc_ldpc = QC_LDPC.from_params(n, p, w)
print(qc_ldpc.getG())
print(qc_ldpc.getG().shape)

In [ ]:
print(qc_ldpc.getH())
print(qc_ldpc.getH().shape)

In [ ]:
word = np.random.randint(2, size=qc_ldpc.getG().shape[0])

In [ ]:
word

In [ ]:
print(len(word))

In [ ]:
encoded = qc_ldpc.encode(word)
print(encoded)
print(len(encoded))

In [ ]:
# error vector size n with t or less errors
errors_num = 2
e = [1 for _ in range(errors_num)] + [0 for _  in range(n - errors_num)]
e = np.array(e)
np.random.shuffle(e)
print(e)
print(len(e))

In [ ]:
corrupted = (encoded + e) % 2
print(corrupted)
print(len(corrupted))

In [ ]:
decoded = qc_ldpc.decode(np.copy(corrupted))
decoded = qc_ldpc.get_message(decoded)
print(decoded)
print(len(decoded))

In [ ]:
word

In [ ]:
try:
    assert (decoded == word).all()
except AssertionError:
    print("The secret message must be forwarded")

In [ ]:
os.chdir('/content/gdrive/MyDrive/HIDE--main/Animal Species')

In [ ]:
ls

In [ ]:
# Specify the folder path where you want to create the empty folders
folder_path = '/content/gdrive/MyDrive/HIDE--main/Animal Species'

# List of folder names
folder_names = ['Starry triggerfish',
'Flat needlefish',
'Verany\'s enope squid',
'Pfeffer\'s enope squid',
'Freshwater bream',
'Freshwater breams nei',
'Atlantic deep-sea lobster',
'Prickly deep-sea lobster',
'Deep-water mud lobster',
'Argentine seabass',
'Koester',
'Spiny cockle',
'European prickly cockle',
'Poorly ribbed cockle',
'Sand cockle',
'Tuberculate cockle',
'Wahoo',
'Goldsilk seabream',
'Twobar seabream',
'Yellowfin seabream',
'Blackhead seabream',
'Surgeonfishes nei',
'Powderblue surgeonfish',
'Monrovia doctorfish',
'Rathbun\'s giant file shell',
'Aviu shrimp',
'Australian paste shrimp',
'Northern mauxia shrimp',
'Tsivakihini paste shrimp',
'Jawla paste shrimp',
'Taiwan mauxia shrimp',
'Akiami paste shrimp',
'Southern mauxia shrimp',
'Alamang shrimp',
'Jembret shrimp',
'Siberian sturgeon',
'Danube sturgeon\'Green sturgeon',
'Adriatic sturgeon',
'Fringebarbel sturgeon',
'Sterlet sturgeon',
'Starry sturgeon',
'Sturgeon',
'White sturgeon',
'Sturgeons nei',
'Glowbelly',
'Glow-bellies, splitfins nei',
'Girdle anemone',
'Beadlet anemone',
'Sea anemones',
'Surf redfish',
'Hooktooth dogfish',
'Antarctic scallop',
'Antarctic knobbed octopus',
'Queen scallop',
'Redmouth grouper',
'Longfin icedevil',
'Spotted eagle ray',
'Mountain mullet',
'Landlady\'s Wig',
'Babberlocks',
'Roundjaw bonefish',
'Bonefish',
'Bonefishes nei',
'Spiny mole crab',
'Bleak',
'Razorbill',
'Soft corals',
'Alexandria pompano',
'African pompano',
'Indian threadfish',
'Fingerprint oyster',
'Shrimp scad',
'Razorbelly scad',
'Blackfin scad',
'Herring scad',
'Long snouted lancetfish',
'Baird\'s slickhead',
'Seaweeds nei',
'Dovekie',
'American alligator',
'African squid',
'Midsize squid',
'Alloteuthis squids nei',
'European common squid',
'Slender tuna',
'Antarctic neosquid',
'Pelagic thresher',
'Thresher sharks nei',
'Bigeye thresher',
'Thresher',
'Blueback shad',
'Alabama shad',
'Allis shad',
'Allis and twaite shads',
'Caspian shad',
'Twaite shad',
'Hickory shad',
'Pontic shad',
'Alewife',
'American shad',
'Shads nei',
'Mutton hamlet',
'Rivulated mutton hamlet',
'Flathead snapping shrimp',
'Teppo snapping shrimp',
'Forceps snapping shrimp',
'Nymph snapping shrimp',
'Red snapping shrimp',
'Daisy snapping shrimp',
'Bigclaw snapping shrimp',
'Armed snapping shrimp',
'Japanese snapping shrimp',
'Bristle snapping shrimp',
'Queensland snapping shrimp',
'Coral snapping shrimp',
'Corsican moss',
'Orange filefish',
'Scribbled leatherjac. filefish',
'Bleeker smoothbelly sardinella',
'Smoothbelly sardinella',
'Spotted sardinella',
'Black bullhead',
'Brown bullhead',
'Atlantic strawberry cockle',
'American sand lance',
'Lesser sand-eel',
'Pacific sandlance',
'Sandeels(=Sandlances) nei',
'Small sandeel',
'Sandlances nei',
'Softhead sea catfish',
'Bocon toadfish',
'Twobar anemonefish',
'Saucer scallop',
'Paper moon scallop',
'Asian moon scallop',
'Climbing perch',
'Antique ark',
'Basket ark',
'Diluvial ark',
'Rusty ark',
'Grand ark',
'Blood cockle',
'Mazatlan\'s ark',
'Many ribbed ark',
'Nodular ark',
'Eared ark',
'Blood ark',
'Reinhart\'s ark',
'Senegal ark',
'Brown ark',
'Anadara clams nei',
'Black ark',
'Bluespotted wrasse',
'Northern wolffish',
'Atlantic wolffish',
'Spotted wolffish',
'Wolffishes(=Catfishes) nei',
'Common teal',
'Eurasian wigeon',
'Mallard',
'Ducks, geese and swans nei',
'Regan\'s anchovy',
'Deep body anchovy',
'Cuban anchovy',
'Short anchovy',
'Eigenmann\'s anchovy',
'Longfinger anchovy',
'Broad-striped anchovy',
'Slender anchovy',
'Big-eye anchovy',
'Shortfinger anchovy',
'Bay anchovy',
'Longnose anchovy',
'Panama anchovy',
'Spicule anchovy',
'Trinidad anchovy',
'Zabaleta anchovy',
'Bigscale anchovy',
'Broadband anchovy',
'Anchoviellas nei',
'Sharpear enope squid',
'Angel squid',
'Three-spot flounder',
'Snakelocks anemone',
'Seagrasses nei',
'European eel',
'Short-finned eel',
'Japanese eel',
'American eel',
'River eels nei',
'Eels, morays, congers nei',
'Leopard scallop',
'Toothless lucine',
'Chacunda gizzard shad',
'Squamose venus',
'Common jingle shell',
'Peruvian jingle shell',
'Anomuran decapods nei',
'Sablefish',
'Daggertooth',
'Greylag goose',
'Pink-footed goose',
'Bean goose',
'Frogfishes nei',
'Indo-Pacific boarfish',
'Blue antimora',
'Slender grouper',
'Black scabbardfish',
'Intermediate scabbardfish',
'Smalleye scabbardfish',
'Small toothed jobfish',
'Rusty jobfish',
'Transparent goby',
'Sea mouse',
'Freshwater drum',
'Cardinal fish',
'Cardinalfishes, etc. nei',
'Saddle grooved macoma',
'Paper tellin',
'Common pelican-foot',
'Serre\'s pelican-foot',
'Japanese sea cucumber',
'Green jobfish',
'Atlantic ghost catshark',
'Brown catshark',
'Hoary catshark',
'Longfin catshark',
'Smallbelly catshark',
'Broadnose catshark',
'Japanese catshark',
'Longnose catshark',
'Longhead catshark',
'Flathead catshark',
'Madeira catshark',
'Ghost catshark',
'Smalleye catshark',
'Largenose catshark',
'Smallfin catshark',
'Spatulasnout catshark',
'Deep-water catshark',
'Broadgill catshark',
'Saldanha catshark',
'Pale catshark',
'South China catshark',
'Spongehead catshark',
'Deep-water catsharks',
'Panama ghost catshark',
'Borneo catshark',
'Black snapper',
'African forktail snapper',
'King penguin',
'Arapaima',
'Black sea urchin',
'Hazelnut ark',
'Mossy ark',
'Indo-Pacific ark',
'Noah\'s ark',
'Chuchoca ark',
'Ark clams nei',
'Ventricose ark',
'Turkey wing',
'Giant sundial',
'Clear sundial',
'Sheepshead',
'Western Atlantic seabream',
'Ocean quahog',
'Rough spanish lobster',
'Small spanish lobster',
'Royal spanish lobster',
'South American fur seal',
'New Zealand fur seal',
'Galapagos fur seal',
'Antarctic fur seal',
'Juan Fernandez fur seal',
'South African fur seal',
'Fur seals nei',
'Guadalupe fur seal',
'Subantarctic fur seal',
'Japanese sandfish',
'Arcuate mussel',
'Grey heron',
'Speckled swimcrab',
'Sand swimcrab',
'Greater argentine',
'Argentine',
'Argentines',
'Argentines nei',
'Kuro shrimp',
'Oil-vessel triton',
'Greater argonaut',
'Argonauts nei',
'Calico scallop',
'Atlantic bay scallop',
'Peruvian calico scallop',
'Pacific calico scallop',
'Soldierbream',
'King soldier bream',
'Arabian sea meagre',
'Southern meagre(=Mulloway)',
'Meagre',
'Meagres nei',
'Carpenter seabream',
'Sea catfishes nei',
'Indian driftfish',
'Brown driftfish',
'Ariommatids nei',
'New Granada sea catfish',
'Giant red shrimp',
'Indian red shrimp',
'Aristeid shrimps nei',
'Arabian red shrimp',
'Blue and red shrimp',
'Purplehead gamba prawn',
'Smooth red shrimp',
'Aristeus shrimps nei',
'Striped red shrimp',
'Stout red shrimp',
'Giant sea catfish',
'Smoothmouth sea catfish',
'Spotted catfish',
'Madagascar sea catfish',
'Guinean sea catfish',
'Kukwari sea catfish',
'Crucifix sea catfish',
'Giant catfish',
'Scaldback',
'Mediterranean scaldfish',
'Ruff',
'Australian salmon',
'Barbeled plunderfishes nei',
'Argentine stiletto shrimp',
'Brine shrimp',
'Brine shrimps nei',
'Pacific asaphis',
'Sea squirts nei',
'Red harpoon',
'Harpoon seaweeds',
'Gillbacker sea catfish',
'Bressou sea catfish',
'Red gurnard',
'Asp',
'Razorback scabbardfish',
'European crayfishes nei',
'Euro-American crayfishes nei',
'Noble crayfish',
'Danube crayfish',
'Red starfish',
'Starfishes nei',
'Spurred turban',
'Rough turbo',
'Spirate turban',
'Rockpile turban',
'Wavy turban',
'Australian spotted catshark',
'Gulf catshark',
'Striate beach clam',
'Australian marbled catshark',
'Coral catshark',
'Kamchatka flounder',
'Arrow-tooth flounder',
'Big-scale sand smelt',
'Silversides(=Sand smelts) nei',
'Hardyhead silverside',
'Geelbek croaker',
'White weakfish',
'Alligator gar',
'Cuban gar',
'Chautard\'s pen shell',
'Brittle pen shell',
'Maura pen shell',
'Comb pen shell',
'Pen shells nei',
'Tuberculate pen shell',
'Flag pen shell',
'Blackmouth croaker',
'Yellowtail scad',
'Gabon shrimp',
'Basket shrimp',
'Koros shrimp',
'Camacuto shrimp',
'Soldier brush shrimp',
'Ekusa shrimp',
'Orange shrimp',
'Periscope shrimp',
'Cholga mussel',
'Australian blackspot catshark',
'Guinean flagfin',
'Common jellyfish',
'Corrugate lucine',
'West coast sole',
'Mud sole',
'Southeast Atlantic soles nei',
'Giant barnacle',
'White-clawed crayfish',
'Antilles bobtail squid',
'Big bottom bobtail squid',
'Big-eyed bobtail squid',
'Bullet tuna',
'Frigate tuna',
'Frigate and bullet tunas',
'Baron mud lobster',
'Tufted duck',
'Greater scaup',
'Small short razor',
'Antique razor clam',
'Maculated ivory whelk',
'Mud ivory whelk',
'Spiral babylon',
'Edible shipworm',
'Bowhead whale',
'Minke whale',
'Antarctic minke whale',
'Sei whale',
'Bryde\'s whale',
'Blue whale',
'Fin whale',
'Grey triggerfish',
'Bluespotted triggerfish',
'Triggerfishes, durgons nei',
'Titan triggerfish',
'Ballia',
'Hairy ark',
'Decussate ark',
'Almond ark',
'Lurid ark',
'Silver barb',
'Barbel',
'White piddock',
'Dilate piddock',
'Manila piddock',
'Pacific mud piddock',
'Truncate barnea',
'Lyre cranch squid',
'Dragonfishes nei',
'Peppermint swimcrab',
'North Atlantic octopus',
'Globose octopus',
'Eaton\'s skate',
'Kerguelen sandpaper skate',
'McCain\'s skate',
'Murray\'s skate',
'Spinetail ray',
'Bathyraja rays nei',
'Deep water sole',
'Deep-sea squid',
'Violet batissa',
'Batoid fishes nei',
'Toadfishes nei',
'Pacuma toadfish',
'Toadfishes, etc. nei',
'Halfround cardita',
'Garfish',
'Needlefishes, etc. nei',
'Roundtail duckbill',
'Squaretail duckbill',
'Tanner\'s deep-water shrimp',
'January octopus',
'Elongate frostfish',
'Bigeye frostfish',
'Slender frostfish',
'Glacier lantern fish',
'Arnoux\'s beaked whale',
'Baird\'s beaked whale',
'Smallfin gonate squid',
'Schoolmaster gonate squid',
'Berryteuthis gonate squids nei',
'Alfonsinos, etc. nei',
'Alfonsino',
'Splendid alfonsino',
'Alfonsinos nei',
'Clams, etc. nei',
'Combtooth blennies',
'Axilspot hogfish',
'Tarry hogfish',
'Diana\'s hogfish',
'Golden-spot hogfish',
'Barred hogfish',
'Blackbar hogfish',
'Leopard fish',
'Brown sandfish',
'Green humphead parrotfish',
'Purple dye murex',
'Horned murex',
'Bogue',
'Polar cod',
'Lefteye flounders nei',
'Leopard flounder',
'Blind shark',
'Ornate arm squid',
'Common arm squid',
'Arm squids nei',
'Bigeye grunt',
'Laulao catfish',
'Marine crabs nei',
'Distant scallop',
'Atlantic pomfret',
'Lesser bream',
'Pacific pomfret',
'Pomfrets, ocean breams nei',
'Tilefishes nei',
'Zebra tilefish',
'Brent goose',
'Canada goose',
'Barnacle goose',
'Unicorn cod',
'Brazilian menhaden',
'Gulf menhaden',
'Argentine menhaden',
'Yellowfin menhaden',
'Menhadens nei',
'Atlantic menhaden',
'Tusk(=Cusk)',
'Bearded brotula',
'Goatsbeard brotula',
'Striated buccinum',
'Humphrey\'s buccinum',
'Whelk',
'Common goldeneye',
'Purse frog shell',
'Dwarf frog shell',
'Common frog shell',
'Miran bullia',
'Warty frog shell',
'Frog shell nei',
'Whelks',
'Blue and gold fusilier',
'Redbelly yellowtail fusilier',
'Lunar fusilier',
'Fusiliers Caesio nei',
'Suez fusilier',
'Yellow and blueback fusilier',
'Variable-lined fusilier',
'Yellowback fusilier',
'Fusiliers nei',
'Spectacled caiman',
'Porgies',
'Nodose box crab',
'Arched box crab',
'Flame box crab',
'Rough box crab',
'Shamefaced crab',
'Ornamented boxcrab',
'Ocellated box crab',
'Spiny box crab',
'Spotted box crab',
'Small arched box crab',
'Yellow box crab',
'Hermit anemone',
'Ghost shrimps nei',
'Bigfisted swimcrab',
'Cuata swimcrab',
'Warrior swimcrab',
'Blunttooth swimcrab',
'Dana swimcrab',
'Rugose swimcrab',
'Masked swimcrab',
'Maracaibo swimcrab',
'Marbled swimcrab',
'Shelling crab',
'Gladiator swimcrab',
'Sharptooth swimcrab',
'Blue crab',
'Lesser blue crab',
'Callinectes swimcrabs nei',
'Giant swimcrab',
'Dragonets nei',
'Sand dragonet',
'Shining top-shell',
'Granular top-shell',
'Gualtieri\'s top-shell',
'Laugier\'s top-shell',
'Painted top-shell',
'Smooth callista',
'Manifold callophyllis',
'Elephantfishes, etc. nei',
'Plownose chimaera',
'Cape elephantfish',
'Ghost shark',
'Northern fur seal',
'Keeled mud lobster',
'Cory\'s shearwater',
'Cape Verde shearwater',
'Carolines parrotfish',
'Spinytooth parrotfish',
'Turreted drill',
'Chinese cup-and-saucer',
'American crayfishes nei',
'Vadigo',
'Sabre prawn',
'Japanese rock crab',
'California red rock crab',
'Toothed rock crab',
'Jonah crab',
'Mola rock crab',
'Atlantic rock crab',
'Northern lemon rock crab',
'Dungeness crab',
'Edible crab',
'Pacific rock crab',
'Jonah crabs, rock crabs nei',
'D\'Orbigny\'s buccinum',
'Waved goblet',
'Filefishes nei',
'Whitespotted filefish',
'Rough triggerfish',
'Pygmy right whale',
'Boarfishes nei',
'Boarfish',
'Hungarian cap-shell',
'Carangids nei',
'Orangespotted trevally',
'Yellowspotted trevally',
'Malabar trevally',
'Yellow jack',
'Green jack',
'Blue runner',
'Longfin crevalle jack',
'Crevalle jack',
'Giant trevally',
'Bluefin trevally',
'False scad',
'Bar jack',
'Bigeye trevally',
'Jacks, crevalles nei',
'Pearlfishes nei',
'Goldfish',
'Crucian carp',
'Requiem sharks nei',
'Blacknose shark',
'Silvertip shark',
'Bignose shark',
'Graceful shark',
'Grey reef shark',
'Pigeye shark',
'Borneo shark',
'Copper shark',
'Spinner shark',
'Nervous shark',
'Whitecheek shark',
'Silky shark',
'Creek whaler',
'Galapagos shark',
'Pondicherry shark',
'Finetooth shark',
'Bull shark',
'Blacktip shark',
'Oceanic whitetip shark',
'Hardnose shark',
'Blacktip reef shark',
'Dusky shark',
'Caribbean reef shark',
'Sandbar shark',
'Smalltail shark',
'Blackspot shark',
'Night shark',
'Spot-tail shark',
'Carcharhinus sharks nei',
'Blacktail reef shark',
'Sand tiger shark',
'Great white shark',
'Mangrove horseshoe crab',
'Mediterranean shore crab',
'Green crab',
'Carcinus crabs nei',
'Cockles nei',
'Ajar cardita',
'Lagoon land crab',
'Mouthless land crab',
'Giant land crab',
'Antique cardita',
'Twotoned cardita',
'Tankerville\'s cardita',
'Costate cockle',
'Gaping cockle',
'Loggerhead turtle',
'African caridina',
'Sawtooth caridina',
'Malagasy caridina',
'Needlenose caridina',
'Smooth caridina',
'Common caridina',
'Bengal caridina',
'Caridina shrimps nei',
'Tonkin caridina',
'Pugnose caridina',
'Batwing coral crab',
'Caspian shads',
'Caspian lamprey',
'Helmets shells nei',
'Spiny bonnet',
'Rugose bonnet',
'Horned helmet',
'Flame helmet',
'Emperor helmet',
'Tessellate helmet',
'King helmet',
'Chilean skua',
'South polar skua',
'Madamango sea catfish',
'Suckers nei',
'Leafy caulerpa',
'Caulerpa seaweeds',
'Atlantic goldeneye tilefish',
'Grey tilefish',
'Ocean whitefish',
'Rayed limpet',
'Turtle limpet',
'Picarels, etc. nei',
'Curled picarel',
'Redfish',
'Red panache',
'Rock cook',
'Emerald wrasse',
'Ruffs, barrelfishes nei',
'Rudderfish',
'Needle dogfish',
'Gulper shark',
'Dumb gulper shark',
'Lowfin gulper shark',
'Smallfin gulper shark',
'Taiwan gulper shark',
'Gulper sharks nei',
'Leafscale gulper shark',
'Mosaic gulper shark',
'Little gulper shark',
'Largescale fat snook',
'Snooks(=Robalos) nei',
'Common snook',
'White snook',
'Black seabass',
'Dusky angelfish',
'Black dogfish',
'Granular dogfish',
'Bareskin dogfish',
'Combtooth dogfish',
'Ornate dogfish',
'Whitefin dogfish',
'Centroscyllium dogfishes nei',
'Portuguese dogfish',
'Longnose velvet dogfish',
'Shortnose velvet dogfish',
'Roughskin dogfish',
'Plunket shark',
'Peacock hind',
'Golden hind',
'Chocolate hind',
'Graysby',
'Bluespotted hind',
'Coney',
'Yellowfin hind',
'Garish hind',
'Leopard hind',
'Freckled hind',
'Coral hind',
'Niger hind',
'Sixblotch hind',
'Tomato hind',
'Strawberry hind',
'Bluespotted seabass',
'Darkfin hind',
'Cephalopods nei',
'Commerson\'s dolphin',
'Black dolphin',
'Heaviside\'s dolphin',
'Hector\'s dolphin',
'Reticulated swellshark',
'Draughtsboard shark',
'Australian swellshark',
'Whitefinned swellshark',
'Indian swellshark',
'Balloon shark',
'Swellshark',
'Lollipop catshark',
'Red bandfish',
'Black guillemot',
'Velvet ceramium',
'Pacific ceramium',
'Common ceramium',
'Common edible cockle',
'Olive green cockle',
'Madeira lantern fish',
'Girdled horn shell',
'Obtuse horn shell',
'Quadrate horn shell',
'Spicate cerithe',
'Coral cerith',
'Spinose cerith',
'Giant knobbed cerith',
'Mediterranean cerithe',
'Common cerithe',
'Blue squat lobster',
'Atlantic anchoveta',
'Pacific anchoveta',
'Basking shark',
'Deep-sea red crab',
'El Dorado shrimp',
'West African geryon',
'Southwest Atlantic red crab',
'Red crab',
'Chaceon geryons nei',
'Blackfin icefish',
'Spiny icefish',
'Hooktooth shark',
'Pacific spadefish',
'Butterflyfishes',
'Hairy chaetomorpha',
'Floating chaetomorpha',
'Mexican jewel box',
'Lazarus jewel box',
'Reflexed jewel box',
'Savigny\'s jewel box',
'Striped venus',
'Mackerel icefish',
'Snakehead',
'Indonesian snakehead',
'Spotted snakehead',
'Snakeheads(=Murrels) nei',
'Striped snakehead',
'Crocodile icefishes nei',
'Unicorn icefish',
'Milkfish',
'Characins nei',
'Knobbed triton',
'Variegated triton',
'Charybdis crabs nei',
'Pelican flounder',
'Floral wrasse',
'Broomtail wrasse',
'Humphead wrasse',
'Castaneta',
'Redfingers',
'Peruvian morwong',
'Margined flyingfish',
'Atlantic flyingfish',
'Guinean flyingfish',
'Bennett’s flyingfish',
'Santer seabream',
'Cape gurnard',
'Bluefin gurnard',
'Tub gurnard',
'Longfin gurnard',
'Lesser gurnard',
'Indo-Pacific gurnards',
'Thicklip grey mullet',
'Largescale mullet',
'Otomebora mullet',
'Goldspot mullet',
'Tade gray mullet',
'Greenback mullet',
'Eastern Pacific green turtle',
'Green turtle',
'West Indian murex',
'Adusta murex',
'Ramose murex',
'Firebrand murex',
'Virginal murex',
'Arabian carpetshark',
'Burmese bambooshark',
'Bluespotted bambooshark',
'Grey bambooshark',
'Hasselt\'s bambooshark',
'Slender bambooshark',
'Whitespotted bambooshark',
'Brownbanded bambooshark',
'Rabbit fish',
'Chimaeras, etc. nei',
'Common Californian venus',
'Small banded venus',
'Smooth Pacific venus',
'Ornate venus',
'Kellett\'s Panama venus',
'Semi-rough venus',
'Frilled Californian venus',
'Ocellated icefish',
'Queen crab',
'Tanner crabs nei',
'Dogtooth herring',
'Dorab wolf-herring',
'Whitefin wolf-herring',
'Wolf-herrings nei',
'Articulate chiton',
'Chitons nei',
'Stoke\'s chiton',
'Frilled shark',
'Iceland scallop',
'Little bay scallop',
'Noble scallop',
'Senatorial scallop',
'Scaly Pacific scallop',
'Variegated scallop',
'Greeneyes',
'Shortnose greeneye',
'Spotted greeneye',
'Green seaweeds',
'Atlantic bumper',
'Pacific bumper',
'Bumpers nei',
'Green shrimp',
'Bleeker\'s parrotfish',
'Robust tuskfish',
'Tuskfishes nei',
'Spurweed',
'Hackle weed',
'Cartilaginous fishes nei',
'Common nase',
'Carragheen (Irish) moss',
'Choro mussel',
'Pinpricked mussel',
'Azores chromis',
'Englishman seabream',
'Daggerhead seabream',
'Red stumpnose seabream',
'Roman seabream',
'False red stumpnose',
'Slinger seabream',
'Daggerhead breams nei',
'Cichlasoma nei',
'Cichlids nei',
'Fivebeard rockling',
'Corvina drum',
'Ciona',
'Script venus',
'Chamber venus',
'African venus',
'Mandarin dogfish',
'Mud carp',
'Hawkfishes nei',
'Stocky hawkfish',
'Barbelthroat carpetshark',
'Taiwan saddled carpetshark',
'Saddle carpetshark',
'Old woman octopus',
'Pacific sanddab',
'Citharids nei',
'Citharinus nei',
'Spotted flounder',
'West Indian top shell',
'Cladodonta',
'Jussieu\'s chanculus',
'Long-tailed duck',
'North African catfish',
'Africa-bighead catfish, hybrid',
'Mangrove hermit crab',
'Clinids fishes nei',
'Mud mantis',
'Chinese gizzard shad',
'Atlantic herring',
'Pacific herring',
'Herrings, sardines nei',
'Diadromous clupeoids nei',
'Clupeoids nei',
'Black and Caspian Sea sprat',
'Black-Caspian Sea sprats nei',
'Circular sea bisquit',
'Necklace cerith',
'Cnidarians nei',
'Interrupted lucine',
'Atlantic tiger lucine',
'Punctate lucine',
'Pacific tiger lucine',
'Fragile codium',
'Hollowsnout grenadier',
'Clearsnout grenadier',
'Shoulderspot grenadier',
'Goldspotted grenadier anchovy',
'Osbeck\'s grenadier anchovy',
'Pacific saury',
'Maculated dwarf triton',
'False abalone',
'Golden anemone',
'European conger',
'Whitespotted conger',
'American conger',
'Argentine conger',
'Conger eels nei',
'Conger eels, etc. nei',
'Cone shells nei',
'Barred grunt',
'Beech cone',
'Crowned cone',
'Yellow Pacific cone',
'General cone',
'Leopard cone',
'Lettered cone',
'Livid cone',
'Marble cone',
'Mouse cone',
'Oak cone',
'Rayed cone',
'Royal cone',
'Suratan cone',
'Tesselate cone',
'Textile cone',
'Copepods',
'Rough coral moss',
'Lamellose coral-shell',
'Short coral-shell',
'Meyendorff\'s coral shell',
'Momo, boke magai, misu coral',
'Aka coral',
'Shiro, white coral',
'Garnet coral',
'Sardinia coral',
'Angel skin coral',
'Midway deep-sea coral',
'Precious corals nei',
'Asian clam',
'Japanese corbicula',
'Manila clam',
'Freshwater clams nei',
'True heart cockle',
'Vendace',
'Lake cisco',
'Lake(=Common) whitefish',
'European whitefish',
'Houting',
'Whitefishes nei',
'Clown coris',
'Queen coris',
'Rainbow wrasse',
'Common dolphinfish',
'Dolphinfishes nei',
'Roundnose grenadier',
'Sculpins nei',
'Channel bull blenny',
'Japanese sand shrimp',
'Alaska shrimp',
'Gray shrimp',
'Common shrimp',
'California shrimp',
'Blacktailed shrimp',
'Bay shrimp',
'Sand shrimp',
'Crangon shrimps nei',
'Crangonid shrimps nei',
'Columbia black oyster',
'Cortez oyster',
'Gasar cupped oyster',
'Pacific cupped oyster',
'Slipper cupped oyster',
'Mangrove cupped oyster',
'Cupped oysters nei',
'American cupped oyster',
'Karanteen seabream',
'Spiny slipper shell',
'Excavated slipper shell',
'American slipper-limpet',
'Slipper-limpet crepidule',
'Goree slipper shell',
'Onyx slipper shell',
'Nail-shaped crepidule',
'Feather stars and sea lilies',
'Crocodiles and alligators nei',
'Australian crocodile',
'Morelet\'s crocodile',
'Nile crocodile',
'New Guinea crocodile',
'Estuarine crocodile',
'Cuban crocodile',
'Siamese crocodile',
'Humpback grouper',
'Red swimcrab',
'Brown cup-and-saucer',
'Freshwater crustaceans nei',
'Marine crustaceans nei',
'Changallo shrimp',
'Harlequin catshark',
'Striated surgeonfish',
'Rough lima',
'Goldsinny-wrasse',
'Grass carp(=White amur)',
'Toothed-fin squid',
'Shadow driftfish',
'Hooked ark',
'African knife shell',
'Leaping bonito',
'Lachner\'s tonguesole',
'Birdbeak burrfish',
'Oriental cyclina',
'Antarctic cardita',
'Cyclope nassa',
'Lumpfishes nei',
'Lumpfish(=Lumpsucker)',
'Curved mushroom coral',
'Disc-fin squids nei',
'Whooper swan',
'Mute swan',
'Aquatile hairy triton',
'Corrugated triton',
'Cuticle-clad triton',
'Angular triton',
'Intermediate hairy triton',
'Black-spotted triton',
'Shortneck triton',
'Nicobar hairy triton',
'Neapolitan triton',
'Common hairy triton',
'Pear triton',
'Black musselcracker',
'Bat volute',
'Pig\'s snout volute',
'Elephant\'s snout volute',
'Marmorate volute',
'Olla volute',
'Neptune\'s volute',
'Volutes nei',
'Carinate rock shell',
'Shoot rock shell',
'Tonguefishes',
'Nigerian tonguesole',
'Ghanian tonguesole',
'Canary tonguesole',
'Guinean tonguesole',
'Senegalese tonguesole',
'Tonguesole nei',
'Dogtooth grenadier',
'Red pike conger',
'Guinean pike conger',
'Acoupa weakfish',
'Peruvian weakfish',
'Jamaica weakfish',
'Spotted weakfish',
'Squeteague(=Gray weakfish)',
'Weakfishes nei',
'Smalltooth weakfish',
'Striped weakfish',
'Green weakfish',
'Bullmouth helmet',
'Cowries nei',
'Gold ring cowrie',
'Arabian cowrie',
'Eyed cowie',
'Boutet\'s cowrie',
'Serpent\'s head cowrie',
'Carnelian cowrie',
'Dragon cowrie',
'Depressed cowrie',
'Eglantine cowrie',
'Eroded cowrie',
'Isabelle cowrie',
'Monster cowrie',
'Lynx cowrie',
'Reticulated cowrie',
'Map cowrie',
'Humpback cowrie',
'Money cowrie',
'Walled cowrie',
'Onyx cowrie',
'Schilders\' cowrie',
'Jester cowrie',
'Mole cowrie',
'Tiger cowrie',
'Tummy cowrie',
'Pacific deer cowrie',
'Cyprinids nei',
'Sheepshead minnow',
'Common carp',
'Japanese flyingfish',
'Marine bindweed',
'Hooded seal',
'Golden Cystoseira',
'Red dory',
'Oriental flying gurnard',
'Flying gurnards nei',
'Flying gurnard',
'Kitefin shark',
'Dana octopus',
'Water fleas',
'Cape petrel',
'Stingrays, butterfly rays nei',
'Whip stingray',
'Southern stingray',
'Longtail stingray',
'Common stingray',
'Stingrays nei',
'Pelagic stingray',
'Birdbeak dogfish',
'Rough longnose dogfish',
'Arrowhead dogfish',
'Deania dogfishes nei',
'Redtail scad',
'Mackerel scad',
'Shortfin scad',
'Japanese scad',
'Round scad',
'Indian scad',
'Scads nei',
'Roughear scad',
'Cloak scallop',
'Flatribbed scallop',
'Red delesseria',
'White whale',
'Dolphins nei',
'Long-beaked common dolphin',
'Common dolphin',
'Leaf oyster',
'Great worm shell',
'Angolan dentex',
'Barnard dentex',
'Canary dentex',
'Congo dentex',
'Common dentex',
'Pink dentex',
'Large-eye dentex',
'Morocco dentex',
'Dentex nei',
'Leather bass',
'Smooth grouper',
'Leatherback turtle',
'Painted sweetlips',
'Small lantern fish',
'Irish mojarra',
'Distorted mushroom coral',
'European seabass',
'Spotted seabass',
'Seabasses nei',
'Whip shrimp',
'Galjoens nei',
'Galjoen',
'Banded galjoen',
'Wedge sole',
'Ocellated wedge sole',
'Sweet-smelling seaweed',
'Olive forked ribbons',
'Forked ribbons',
'Red seabroom',
'Fleshy dilsea',
'Longspined porcupinefish',
'Spot-fin porcupinefish',
'Globefish, porcupine fish',
'Lister\'s keyhole limpet',
'Amsterdam Island albatross',
'Antipodean albatross',
'Tristan albatross',
'Southern royal albatross',
'Wandering albatross',
'Gibson\'s albatross',
'Northern royal albatross',
'Albatrosses nei',
'Sand perch',
'Highfin sand perch',
'Annular seabream',
'Senegal seabream',
'Zebra seabream',
'Spottail seabream',
'Red Sea seabream',
'Two-banded seabream',
'Sharpsnout seabream',
'White seabream',
'Sargo breams nei',
'Common two-banded seabream',
'Striped escolar',
'Mottled fusilier',
'Parin\'s spinyfish',
'Longwing spinyfin',
'Patagonian toothfish',
'Antarctic toothfish',
'Antarctic toothfishes nei',
'Common distorsio',
'Decussate distorsio',
'Reticulate distorsio',
'Shoulderblade sea cat',
'Corneous wedge clam',
'Rough butterfly donax',
'Californian donax',
'Sculptured donax',
'Cuneate donax',
'Goolwa donax',
'Common Caribbean donax',
'Toothed donax',
'Pacific bean donax',
'Bean donax',
'Gracile donax',
'Fleshy donax',
'Beautiful donax',
'Punctate donax',
'Rugose donax',
'Leather donax',
'Half-striated donax',
'Donax clams',
'Striate donax',
'Truncate donax',
'Smooth donax',
'Lovely donax',
'American gizzard shad',
'Jumbo flying squid',
'Dunker\'s dosinia',
'Mature dosinia',
'Smooth dosinia',
'Dosinia clam',
'Ponderous dosinia',
'Dosinias nei',
'Shango dragonet',
'African sicklefish',
'Spotted sicklefish',
'Sicklefishes nei',
'Redeye sponge crab',
'Sleepy crab',
'Dugong',
'Bull kelp',
'Rainbow sardine',
'Slender rainbow sardine',
'Rainbow sardines nei',
'Rugose mactra',
'Suckerfishes, remoras nei',
'Live sharksucker',
'Echinoderms',
'Sea urchins, etc. nei',
'Purple Pacific sea urchin',
'Bramble shark',
'Prickly shark',
'European edible sea urchin',
'Unarmed cranch squid',
'Malabar sprat',
'Arboreal ferret',
'Rainbow runner',
'Deep-water sharks nei',
'Sharks, rays, skates, etc. nei',
'Electric lantern fish',
'Horned octopus',
'Combed octopus',
'Musky octopus',
'Horned and musky octopuses',
'Patagonian blennie',
'Saffron cod',
'Navaga(=Wachna cod)',
'Gudgeons, sleepers nei',
'E. Asian fourfinger threadfin',
'Fourfinger threadfin',
'Threefinger threadfin',
'Judas ear cassidula',
'Midas ear cassidula',
'West African ladyfish',
'Ladyfish',
'Senegalese ladyfish',
'Coldwater mole crab',
'Tropical mole crab',
'Bonnetmouths, rubyfishes nei',
'Cape bonnetmouth',
'Fangtooth moray',
'Viper moray',
'Fourbeard rockling',
'Giant keyhole sand dollar',
'Devis\' anchovy',
'Shorthead anchovy',
'Buccaneer anchovy',
'Anchovies, etc. nei',
'Argentine anchovy',
'Southern African anchovy',
'European anchovy',
'Japanese anchovy',
'Californian anchovy',
'Anchoveta(=Peruvian anchovy)',
'Anchovies nei',
'Largescale flounder',
'Sea otter',
'Atl.jackknife(=Atl.razor clam)',
'Pod razor shell',
'Goree razor clam',
'Sword razor shell',
'Ensis razor clams nei',
'Patagonian giant octopus',
'Bright green nori',
'Flat green nori',
'Hollow green nori',
'Welded green nori',
'Dark green nori',
'Northern California lamprey',
'Kern brook lamprey',
'Pit–Klamath brook lamprey',
'Vancouver lamprey',
'Miller Lake lamprey',
'Klamath lamprey',
'Pacific lamprey',
'Petrale sole',
'Spadefishes nei',
'Divergent lucine',
'Cardinal fishes nei',
'Black cardinal fish',
'Rooster hind',
'Rock hind',
'White grouper',
'Hong Kong grouper',
'White-edged grouper',
'Banded grouper',
'Spotted grouper',
'Catface grouper',
'Areolate grouper',
'Yellow grouper',
'Duskytail grouper',
'Longtooth grouper',
'Whitespotted grouper',
'Dogtooth grouper',
'Moustache grouper',
'Brownspotted grouper',
'Olive grouper',
'Orange-spotted grouper',
'Goldblotch grouper',
'Speckled blue grouper',
'Saddletail grouper',
'Spinycheek grouper',
'Speckled hind',
'Dotted grouper',
'Rock grouper',
'Blacktip grouper',
'Barred-chest grouper',
'Blue-and-yellow grouper',
'Yellowedge grouper',
'Brown-marbled grouper',
'Dungat grouper',
'Red hind',
'Haifa grouper',
'Bridled grouper',
'Starspotted grouper',
'Marquesan grouper',
'Jewfish',
'Starry grouper',
'Giant grouper',
'Striped grouper',
'Longspine grouper',
'Snubnose grouper',
'Highfin grouper',
'Speckled grouper',
'Malabar grouper',
'Dusky grouper',
'One-blotch grouper',
'Honeycomb grouper',
'Netfin grouper',
'Red grouper',
'Comet grouper',
'White-blotched grouper',
'Misty grouper',
'Warsaw grouper',
'Snowy grouper',
'Eightbar grouper',
'Dot-dash grouper',
'Smallscaled grouper',
'Camouflage grouper',
'Hawaiian grouper',
'Longfin grouper',
'Oblique-banded grouper',
'Red-tipped grouper',
'Halfmoon grouper',
'Convict grouper',
'Sixbar grouper',
'Foursaddle grouper',
'Groupers nei',
'Black-dotted grouper',
'Epaulet grouper',
'Nassau grouper',
'Summan grouper',
'Greasy grouper',
'Threespot grouper',
'Reticulate grouper',
'Potato grouper',
'Wavy-lined grouper',
'Domine',
'Slender ponyfish',
'Hawksbill turtle',
'Cuban ribbontail catshark',
'Pygmy ribbontail catshark',
'African ribbontail catshark',
'Bearded seal',
'Hair crab',
'Chinese mitten crab',
'Warty crab',
'Red sea mantis shrimp',
'Gray whale',
'White sardine',
'Northern pike',
'Amur pike',
'Olive ark',
'Deep-water red snapper',
'Deepwater longtail red snapper',
'Queen snapper',
'Pale snapper',
'Fantail darter',
'Bonga shad',
'Pacific menhaden',
'Giant ciliate Ethusa',
'Shorttail lanternshark',
'Lined lanternshark',
'Combtoothed lanternshark',
'Broadbanded lanternshark',
'Southern lanternshark',
'Carribean lanternshark',
'Blackbelly lanternshark',
'African lanternshark',
'Great lanternshark',
'Smooth lanternshark',
'Fringfin lanternshark',
'Thorny lanternshark',
'Velvet belly',
'Lanternsharks nei',
'Brown lanternshark',
'Hawaiian lanternshark',
'Green lanternshark',
'Red-eye round herring',
'Whitehead\'s round herring',
'Yamato shrimp',
'Greenland shrimp',
'Iso shrimp',
'Australian crayfish',
'Map octopus',
'Southern right whale',
'Northern right whale',
'North Pacific right whale',
'Spiny eucheuma',
'Pacific flagfin mojarra',
'Flagfin mojarra',
'Tasselled wobbegong',
'Carpathian lamprey',
'Greek brook lamprey',
'Ukrainian brook lamprey',
'Korean lamprey',
'Rockhopper penguin',
'Macaroni penguin',
'Flying halfbeak',
'Ribbon halfbeak',
'Fourline snakeblenny',
'Steller sea lion',
'Red lobster',
'Sculptured lobster',
'Banded lobster',
'Ice krill',
'Pygmy krill',
'Antarctic krill nei',
'Antarctic krill',
'Spiny krill',
'Northern krill',
'Pelagic swimcrab',
'Robustus swimcrab',
'Longtooth hairtail',
'Smallhead hairtail',
'Taillight shark',
'Pygmy shark',
'Humming-bird bobtail squid',
'Mimika bobtail squid',
'Southern bobtail squid',
'Kawakawa',
'Little tunny(=Atl.black skipj)',
'Black skipjack',
'Grey gurnard',
'Shield fan lobster',
'Poey\'s scabbardfish',
'Crimson seabream',
'Marine shells nei',
'Pearl oyster shells nei',
'Trochus shells nei',
'Turban shells nei',
'Freshwater mussel shells',
'Hunter shrimp',
'Companion shrimp',
'Cock shrimp',
'Flyingfishes nei',
'Cipango prawn',
'Ridgetail prawn',
'Mamtom prawn',
'Siberian prawn',
'Oriental prawn',
'Roshna prawn',
'Wooden fasciolaria',
'True tulip',
'Pygmy killer whale',
'Graceful fig shell',
'Underlined fig shell',
'Sevenfinger threadfin',
'Javanese threadfin',
'Splendid threadfin',
'Eightfinger threadfin',
'Indian sevenfinger threadfin',
'Yellowthread threadfin',
'Common basket lucina',
'Elegant basket lucina',
'Barbados keyhole limpet',
'Compressed keyhole limpet',
'Rayed keyhole limpet',
'Cloudy keyhole limpet',
'Green Panama keyhole limpet',
'Bluespotted cornetfish',
'Pacific cornetfish',
'Red cornetfish',
'Cornetfish',
'Flexuous scallop',
'Smooth scallop',
'Proteus scallop',
'White strawberry cockle',
'Pacific half cockle',
'Pacific strawberry cockle',
'Atlantic puffin',
'Black-bellied storm petrel',
'Wracks nei',
'Bladder wrack',
'Adriatic wrack',
'Northern fulmar',
'Southern fulmar',
'Paper cockle',
'Red forkweed',
'Whiskery shark',
'Philippi\'s spindle',
'Distaff spindle',
'Du Petit\'s spindle',
'Nicobar spindle',
'Beaked fusus',
'Syracusan fusus',
'Silvery pout',
'Gadiformes nei',
'Pacific cod',
'Atlantic cod',
'Greenland cod',
'Northern cods nei',
'Discrepant venus',
'Forked venus',
'Comb venus',
'Tumid venus',
'Mediterranean bigeye rockling',
'Threadfin rockling',
'Shore rockling',
'Rocklings nei',
'Three-bearded rockling',
'Common galatea clam',
'Craylets, squat lobsters',
'White barbel',
'Tiger shark',
'Lesser African threadfin',
'Tope shark',
'Roughtail catshark',
'Australian sawtail catshark',
'Gecko catshark',
'Blackmouth catshark',
'Mouse catshark',
'Broadfin sawtail catshark',
'Peppered catshark',
'African sawtail catshark',
'Blacktip sawtail catshark',
'Dwarf sawtail catshark',
'Crest-tail catsharks nei',
'Armed cranch squid',
'Ice-cube tray coral',
'Depressed sunset clam',
'Elongate sunset clam',
'Faeroe sunset clam',
'Sunset clams nei',
'Squamose sunset clam',
'Courtesan sunset clam',
'Truncate sunset clam',
'Butterfly kingfish',
'Three-spined stickleback',
'Sticklebacks',
'Fragile tellin',
'Gastropods nei',
'White-billed diver',
'Black-throated diver',
'Great northern diver',
'Red-throated diver',
'Toothpony',
'Malpelo land crab',
'Red Pacific land crab',
'Whitespotted land crab',
'Chaffweed',
'Giant gelidium',
'Hairy gelidium',
'Red gelidium',
'Dwarf gelidium',
'Gelidium seaweeds',
'Snake mackerels, escolars nei',
'Snake mackerel',
'White croaker',
'Pink cusk-eel',
'Kingklip',
'Red cusk-eel',
'Black cusk-eel',
'Cusk-eels nei',
'Pouched lamprey',
'Mojarras, etc. nei',
'Saddleback silver-biddy',
'Longtail silverbiddy',
'Guinean striped mojarra',
'Slender silver-biddy',
'Common silver-biddy',
'Mojarras(=Silver-biddies) nei',
'Mediterranean geryon',
'Geryons nei',
'Deep-sea crabs, geryons nei',
'Adanson\'s gibbula',
'White gibbula',
'Red-brown gibbula',
'Divaricate gibbula',
'Chapel gibbula',
'Magus gibbula',
'Philbert\'s gibbula',
'Concave gibbula',
'Richard\'s gibbula',
'Umbilicate medit. gibbula',
'Variegate gibbula',
'Skottsberg\'s gigartina',
'Gigartina seaweeds nei',
'Short-tail nurse shark',
'Nurse shark',
'Nurse sharks nei',
'Opaleye',
'Gulf opal eye',
'Greenish glauconomya',
'Short-finned pilot whale',
'Long-finned pilot whale',
'Royal cloak scallop',
'Deep-sea smelt',
'Oxheart cockle',
'Two-spotted bittersweet',
'Giant bittersweet',
'Common European bittersweet',
'Speckled bittersweet',
'Pilose bittersweet',
'Reeve\'s bittersweet',
'Scripta bittersweet',
'Violet bittersweet',
'Vovan bittersweet',
'Ganges shark',
'Speartooth shark',
'Armoured shrimp',
'Panama armoured shrimp',
'Spiny armoured shrimp',
'Vicary armoured shrimp',
'Kangaroo shrimp',
'Witch flounder',
'Rex sole',
'Golden trevally',
'Clingfishes nei',
'Freshwater gobies nei',
'Gobies nei',
'Bellotti\'s goby',
'Bucchich\'s goby',
'Giant goby',
'Red-mouthed goby',
'Sarato\'s goby',
'Slender goby',
'Black goby',
'Rock goby',
'Roule\'s goby',
'Atlantic gobies nei',
'Striped goby',
'Golden goby',
'Sailback houndshark',
'Slender smooth-hound',
'Equilateral venus',
'Green birdmouth wrasse',
'Gonate squids nei',
'Boreopacific gonate squid',
'Mako gonate squid',
'Boreoatlantic gonate squid',
'Shortarm gonate squid',
'Madokai gonate squid',
'Atlantic gonate squid',
'Spanish flag',
'Purple mangrove crab',
'Racer mangrove crab',
'Bristlemouths',
'Masked grouper',
'Leather gracilaria',
'Spiny gracilaria',
'Gracilaria seaweeds',
'Broadleaf gracilaria',
'Veiled gracilaria',
'Warty gracilaria',
'Shark mackerel',
'Double-lined mackerel',
'Spotfin flathead',
'Risso\'s dolphin',
'Lightfoot crab',
'Mediterranean sand eel',
'Smooth sandeel',
'Gymnammodytes sandeels nei',
'Slender fusilier',
'Ruffe',
'Japanese large-eye bream',
'Blue-lined large-eye bream',
'Grey large-eye bream',
'Dogtooth tuna',
'Dark moray',
'Canary moray',
'Spotjaw moray',
'Starry moray',
'Brown moray',
'Banded knifefish',
'Spiny butterfly ray',
'California butterfly ray',
'Butterfly rays nei',
'Crested murex',
'Trident shrimp',
'Grunts, sweetlips nei',
'White margate',
'White grunt',
'Yellowstripe grunt',
'Raucous grunt',
'Arabian catshark',
'Speckled catshark',
'Blackspotted catshark',
'Dusky catshark',
'New Zealand catshark',
'Bristly catshark',
'Spotless catshark',
'Lined catshark',
'Mud catshark',
'Tiger catshark',
'Quagga catshark',
'White-tailed sea-eagle',
'Checkerboard wrasse',
'Dusky wrasse',
'Grey seal',
'Sea cactus',
'Donkey\'s ear abalone',
'Threaded abalone',
'Pink abalone',
'Black abalone',
'Japanese abalone',
'Southern green abalone',
'Giant abalone',
'Glistening abalone',
'Perlemoen abalone',
'Oval abalone',
'Planate abalone',
'Blacklip abalone',
'Red abalone',
'White abalone',
'Abalones nei',
'Tuberculate abalone',
'Variable abalone',
'Chilean knife shrimp',
'Jack-knife shrimp',
'Knife shrimps nei',
'Knife shrimp',
'Blue petrel',
'Lusitanian toadfish',
'Halosaurs nei',
'Puffadder shyshark',
'Brown shyshark',
'Dark shyshark',
'False herring',
'Redear herring',
'Scaled herring',
'Scaled sardines',
'Pacific flatiron herring',
'Articulate harp',
'True harp',
'Major harp',
'Bombay-duck',
'Spiny plunderfishes nei',
'Harp shells nei',
'Charcot\'s volute',
'Longnose chimaeras',
'Glassy mactra',
'Sandbeach auger',
'Goldmouth murex',
'Don Moore\'s murex',
'Snipe\'s bill murex',
'Messorius murex',
'Bent-beak murex',
'Velvet helcion',
'Blackbelly rosefish',
'Rosefishes nei',
'Kissing gourami',
'Bluntnose jack',
'Bicolor jack',
'Yellowfin jack',
'Blackfin jack',
'Banded jewelfish',
'Common hemidonax',
'Sicklefin weasel shark',
'Blackeye thicklip',
'Halfbeaks nei',
'Jumping halfbeak',
'Balao halfbeak',
'Ballyhoo halfbeak',
'Indonesia speckled carpetshark',
'Papuan epaulette shark',
'Epaulette shark',
'Hooded carpetshark',
'Speckled carpetshark',
'Keeled witch mantis',
'Japanese topeshark',
'Whitefin topeshark',
'Masked bannerfish',
'Globose box crab',
'Havana box crab',
'Spoted box crab',
'Flecked box crab',
'Shortspine shrimp',
'Toy shrimp',
'Flexed shrimp',
'Tsuno shrimp',
'Sharpnose sevengill shark',
'Spotback herring',
'Bluestripe herring',
'Reunion herring',
'African catfish',
'Dorodotes shrimp',
'Three-spined nylon shrimp',
'Madagascar nylon shrimp',
'Armed nylon shrimp',
'Humpback nylon shrimp',
'Panama nylon shrimp',
'Smooth nylon shrimp',
'Chilean nylon shrimp',
'Mino nylon shrimp',
'Nylon shrimps nei',
'Scarred nylon shrimp',
'Northern nylon shrimp',
'Indian nylon shrimp',
'Western small false donax',
'Horn shark',
'Crested bullhead shark',
'Japanese bullhead shark',
'Mexican hornshark',
'Port Jackson shark',
'Galapagos bullhead shark',
'Whitespotted bullhead shark',
'Zebra bullhead shark',
'True sole',
'Bluegray carpetshark',
'Longnose pygmy shark',
'Odd bobtail squid',
'Bluntnose sixgill shark',
'Ambiguous murex',
'Angular murex',
'Endive murex',
'Black murex',
'Prince murex',
'Radish murex',
'Kelee shad',
'Thickened nassa',
'Filed nassa',
'Netted nassa',
'Giant rock scallop',
'Pacific mole crab',
'Striated mole crab',
'Long-snouted seahorse',
'Short snouted seahorse',
'Flathead sole',
'Amer. plaice(=Long rough dab)',
'Atlantic halibut',
'Pacific halibut',
'Bear paw clam',
'China clam',
'Honey comb',
'Fourwing flyingfish',
'Bony flyingfish',
'Umbrella squid',
'Flowervase jewell squid',
'Elongate jewell squid',
'Reverse jewell squid',
'Squirrelfishes nei',
'African spotted catshark',
'Izak catshark',
'Snakefish sea cucumber',
'Pinkfish',
'White teatfish',
'Elephant trunkfish',
'Bottleneck sea cucumber',
'White threads fish',
'Black teatfish',
'Sea cucumbers nei',
'Giant stone crab',
'Blood-stained turbo',
'American lobster',
'Cape lobster',
'European lobster',
'Homarus lobsters nei',
'Homole crab',
'Mexican barred snapper',
'Orange roughy',
'Mediterranean slimehead',
'Smallscale slimehead',
'Huchen',
'Beluga',
'Glassy flying squid',
'Atlantic lyre crab',
'European storm-petrel',
'Spotted ratfish',
'Ratfishes nei',
'Hydrozoans',
'Leopard seal',
'Bermuda lantern fish',
'Veiled shrimp',
'Doris veiled shrimp',
'Hymenopenaeus shrimps nei',
'Wrinkled oyster',
'Honeycomb oyster',
'Folded lagoon cockle',
'Bluenose warehou',
'Northern bottlenose whale',
'Southern bottlenose whale',
'Great sandeel',
'Crozier weed',
'Tattered sea moss',
'Robust sea moss',
'Samoan silverside',
'Blacktipe tope',
'Pond smelt',
'Surf smelt',
'Silver carp',
'Bighead carp',
'Silver, bighead carps nei',
'Highwaterman catfish',
'Congaturi halfbeak',
'African halfbeak',
'Quoy\'s garfish',
'California halfbeak',
'Japanese halfbeak',
'Common halfbeak',
'Sevenbar grouper',
'Longnose houndshark',
'Bigeye houndshark',
'Velvet fan lobster',
'Japanese fan lobster',
'Smooth fan lobster',
'Butterfly fan lobster',
'Ohio lamprey',
'Chestnut lamprey',
'Northern brook lamprey',
'Southern brook lamprey',
'Mountain brook lamprey',
'Silver lamprey',
'Channel catfish',
'Catfishes nei',
'Bigmouth buffalo',
'Buffalofishes nei',
'Loppe\'s tadpole fish',
'West African ilisha',
'Elongate ilisha',
'Pacific ilisha',
'Bigeye ilisha',
'Indian ilisha',
'Argentine shortfin squid',
'Broadtail shortfin squid',
'Northern shortfin squid',
'Sharptail shortfin squid',
'Shortfin squids nei',
'Longman\'s beaked whale',
'Boto',
'Two-spot razorfish',
'Peacock wrasse',
'Fivefinger wrasse',
'Aquatic invertebrates nei',
'Tall false donax',
'Giant false donax',
'Delessert\'s false donax',
'Smooth false donax',
'Rostrate false donax',
'Iridea',
'Iridea nei',
'Cabinza grunt',
'Cookie cutter shark',
'Largetooth cookiecutter shark',
'Saddle tree oyster',
'Wader tree oyster',
'Janus tree oyster',
'Rayed tree oyster',
'Pacific tree oyster',
'Daggernose shark',
'Bigtooth corvina',
'Giant sea cucumber',
'Marlins,sailfishes,etc. nei',
'Atlantic sailfish',
'Indo-Pacific sailfish',
'Shortfin mako',
'Longfin mako',
'Mako sharks',
'Southern spider crab',
'Pacific coral moss',
'Fine coral moss',
'Juan Fernandez rock lobster',
'Cape rock lobster',
'St.Paul rock lobster',
'Rock lobsters nei',
'Tristan da Cunha rock lobster',
'Green rock lobster',
'Dwarf round herring',
'Sharpnose hammer croaker',
'Karut croaker',
'Bobo mullet',
'Exasperating jujubine',
'West Indian furrow lobster',
'Gibbon furrow lobster',
'Hiant venus',
'Japan venus',
'Marbled venus',
'Giant stargazer',
'Skipjack tuna',
'Tuberculate kneefastia',
'Pygmy sperm whale',
'Dwarf sperm whale',
'Dotted gizzard shad',
'Sea chubs nei',
'Blue sea chub',
'Bermuda sea chub',
'Kyphosus sea chubs nei',
'Orangefin labeo',
'Roho labeo',
'Rhinofishes nei',
'Wrasses, hogfishes, etc. nei',
'Ballan wrasse',
'Brown wrasse',
'Cuckoo wrasse',
'Labrus wrasses nei',
'Green wrasse',
'Hogfish',
'False trevally',
'Norwegian egg cockle',
'Giant Pacific egg cockle',
'Common egg cockle',
'Oblong egg cockle',
'Fraser\'s dolphin',
'Atlantic white-sided dolphin',
'White-beaked dolphin',
'Peale\'s dolphin',
'Hourglass dolphin',
'Pacific white-sided dolphin',
'Dusky dolphin',
'Oceanic puffer',
'Silver-cheeked toadfish',
'Half-smooth golden pufferfish',
'Chiragra spider conch',
'Orange spider conch',
'Common spider conch',
'Milleped spider conch',
'Scorpio spider conch',
'Giant spider conch',
'Tangle',
'Japanese kelp',
'Sea belt',
'Kelps nei',
'Broadfin shark',
'Salmon shark',
'Porbeagle',
'Mackerel sharks,porbeagles nei',
'Oriental worm-eel',
'Hector\'s lanternfish',
'Jewel lanternfish',
'Least brook lamprey',
'Western American river lamprey',
'River lamprey',
'Turkish brook lamprey',
'Pacific brook lamprey',
'European brook lamprey',
'Western brook lamprey',
'Opah',
'Legless cuskeel',
'Large yellow croaker',
'Herring gull',
'Laughing gull',
'Audouin\'s gull',
'Yellow-legged gull',
'Mew gull',
'Kelp gull',
'Lesser black-backed gull',
'Iceland gull',
'Glaucous gull',
'Great black-backed gull',
'Silver gull',
'Common black-headed gull',
'Sabine\'s gull',
'Japanese seabass',
'Antarctic soft-shell clam',
'Truncate lantern clam',
'Barramundi(=Giant seaperch)',
'Nile perch',
'Freshwater perches nei',
'Babel\'s latiaxis',
'Precious stone shell',
'Brown-lined latirus',
'Many-angled spindle',
'Hoso shrimp',
'Medusa shrimp',
'Platenose shrimp',
'Flatnose shrimp',
'Blunt laurenzia',
'Pacific laurenzia',
'Swollen laurenzia',
'Leach\'s cranch squid',
'Ponyfishes(=Slipmouths) nei',
'Orangefin ponyfish',
'Shortnose ponyfish',
'Whipfin ponyfish',
'Ornate ponyfish',
'Ponyfishes(=Slipmouths)',
'Spot croaker',
'Lepadidae barnacles nei',
'Goose barnacle',
'Goose barnacles nei',
'Kemp\'s ridley turtle',
'Olive ridley turtle',
'Escolar',
'Toad mole crab',
'Rock sole',
'Silver scabbardfish',
'Doubtful scabbardfish',
'Pacific scabbardfish',
'Four-spot megrim',
'Megrims nei',
'Megrim',
'Bullhorn gurnard',
'Scalebreast gurnard',
'Spiny gurnard',
'Scalybreast gurnard',
'Gars nei',
'Spotted gar',
'Longnose gar',
'Pumpkinseed',
'Bluegill',
'Shortbeard cusk-eel',
'Specklefin cusk-eel',
'Slender worm-eel',
'Hoven\'s carp',
'Ganges delta prawn',
'Bombay prawn',
'Barbeled houndshark',
'Lesser glass shrimp',
'Indian threadfin',
'Weddell seal',
'Leaf coral',
'Coromandel hairtail',
'Savalai hairtail',
'Southern brown kelp',
'Alaskan brook lamprey',
'American brook lamprey',
'Arctic lamprey',
'Siberian lamprey',
'Western Transcaucasian lamprey',
'Far Eastern brook lamprey',
'Lombardy brook lamprey',
'Emperors(=Scavengers) nei',
'Atlantic emperor',
'Redaxil emperor',
'Yellowtail emperor',
'Orange-spotted emperor',
'Thumbprint emperor',
'Pink ear emperor',
'Sky emperor',
'Smalltooth emperor',
'Trumpet emperor',
'Spangled emperor',
'Orange-striped emperor',
'Ornate emperor',
'Yellowlip emperor',
'Orfe(=Ide)',
'Common dace',
'Chestnut latirus',
'Leerfish',
'Bigeye cranch squid',
'Atlantic piquitinga',
'Pacific piquitinga',
'Spiny file shell',
'Common file shell',
'Yellowfin sole',
'Yellowtail flounder',
'Common dab',
'Limandas nei',
'Inflated file shell',
'Antarctic airy limopsis',
'Horseshoe crab',
'Spear lobster',
'Oriental spear lobster',
'Spear lobsters nei',
'Arched swimming crab',
'Wrinkled swimcrab',
'Blue-leg swimcrab',
'Liocarcinus swimcrabs nei',
'Smooth swimcrab',
'Camp pitar venus',
'Ornate pitar venus',
'Snailfishes nei',
'Wellington shrimp',
'Tang\'s snapper',
'Baiji',
'Northern right whale dolphin',
'Southern right whale dolphin',
'Golden king crab',
'King crab',
'Stone king crab',
'Subantarctic stone crab',
'Southern king crab',
'King crabs nei',
'King crabs, stone crabs nei',
'West coast seabream',
'White steenbras',
'Sand steenbras',
'Steenbrasses nei',
'Pacific date mussel',
'European date mussel',
'Cylinder date mussel',
'Stone weed',
'Carved star-shell',
'Imbricated star-shell',
'Green star-shell',
'Rough periwinkle',
'Common periwinkle',
'Periwinkles nei',
'Diamond mullet',
'Golden grey mullet',
'Sicklefin mullet',
'Largescaled mullet',
'So-iny (redlip) mullet',
'Thinlip grey mullet',
'Leaping mullet',
'Squaretail mullet',
'Dofleini\'s lantern fish',
'Crabeater seal',
'Pacific tripletail',
'Tripletail',
'Inshore squids nei',
'Various squids nei',
'Beka squid',
'Spear squid',
'Mitre squid',
'Indian squid',
'Swordtip squid',
'Veined squid',
'Patagonian squid',
'Japanese squid',
'Kobi squid',
'Bigeye inshore squid',
'Opalescent inshore squid',
'Longfin squid',
'Slender inshore squid',
'Cape Hope squid',
'Island inshore squid',
'Sao Paulo squid',
'Siboga squid',
'Common squids nei',
'Surinam squid',
'Little squid',
'European squid',
'Dart squid',
'Western Atlantic brief squid',
'Guinean thumbstall squid',
'Panama brief squid',
'Thumbstall squids nei',
'Cock\'s comb oyster',
'Anglerfishes nei',
'Longspine African angler',
'Smooth angler',
'Indian turrid',
'American angler',
'Blackbellied angler',
'Blackfin goosefish',
'Yellow goosefish',
'Angler(=Monk)',
'Monkfishes nei',
'Shortspine African angler',
'Devil anglerfish',
'Great Northern tilefish',
'Pale lucina',
'Burbot',
'Giant owl limpet',
'File limpet',
'Chilean sea urchin',
'Sliteye shark',
'Northern lucina',
'Brown moon-shell',
'Guillemin\'s moon-shell',
'Pretty moon-shell',
'Snappers, jobfishes nei',
'Yellow-banded snapper',
'African red snapper',
'Mutton snapper',
'Schoolmaster snapper',
'Mullet snapper',
'Mangrove red snapper',
'Yellow snapper',
'Bengal snapper',
'Two-spot red snapper',
'Blackfin snapper',
'Northern red snapper',
'Blueline snapper',
'Colorado snapper',
'Cubera snapper',
'Checkered snapper',
'African brown snapper',
'Blackspot snapper',
'Guinea snapper',
'Crimson snapper',
'Golden African snapper',
'Dory snapper',
'Blacktail snapper',
'Humpback red snapper',
'Papuan black snapper',
'Gorean snapper',
'Grey snapper',
'Spotted rose snapper',
'Dog snapper',
'John\'s snapper',
'Jordan\'s snapper',
'Common bluestripe snapper',
'Yellowstreaked snapper',
'Lunartail snapper',
'Bigeye snapper',
'Mahogany snapper',
'Malabar blood snapper',
'One-spot snapper',
'Pacific dog snapper',
'Pacific red snapper',
'Southern red snapper',
'Five-lined snapper',
'Blubberlip snapper',
'Russell\'s snapper',
'Humphead snapper',
'Emperor red snapper',
'Snappers nei',
'Star snapper',
'Lane snapper',
'Timor snapper',
'Brownstripe red snapper',
'Silk snapper',
'Marine otter',
'Narrow otter shell',
'Common otter shell',
'Oblong otter shell',
'Atlantic sabretooth anchovy',
'Greater eelpout',
'Eelpouts',
'Siamese shipworm',
'Lion\'s paw',
'Pacific lion\'s paw',
'Queen lizard mantis',
'Lizard mantis',
'Striped mantis',
'Smooth mantis shrimp',
'Lined shrimp',
'Monaco shrimp',
'Indian lined shrimp',
'Black and white snapper',
'Cancellate panope',
'Grand macoma',
'Bentnose macoma',
'White sand macoma',
'Cinnamon river prawn',
'Noumea river prawn',
'Amazonian river prawn',
'Cauque river prawn',
'Koua river prawn',
'Birma river prawn',
'New Caledonia river prawn',
'Painted river prawn',
'Ganges river prawn',
'Kaira river prawn',
'Congo river prawn',
'Rough river prawn',
'Sweet river prawn',
'Crane river prawn',
'Greybeard river prawn',
'Hawaii river prawn',
'Hancock\'s river prawn',
'Cascade river prawn',
'Orana river prawn',
'Slender river prawn',
'Striped river prawn',
'Jaro river prawn',
'Java river prawn',
'Agar river prawn',
'Kuncho river prawn',
'Philippine river prawn',
'Riceland prawn',
'Monkey river prawn',
'Scissor river prawn',
'Mountain river prawn',
'Malayam scale prawn',
'Madagascar scale prawn',
'Brackish river prawn',
'Monsoon river prawn',
'Knobtooth prawn',
'Shortleg river prawn',
'Oriental river prawn',
'Western river prawn',
'Ohio river prawn',
'Buchura river prawn',
'Patsa river prawn',
'Muff prawn',
'Volta river prawn',
'Shortfinger river shrimp',
'Giant river prawn',
'Hairy river prawn',
'Goda river prawn',
'Sunda river prawn',
'River prawns nei',
'Suriname river prawn',
'Longarm river prawn',
'Forest river prawn',
'Dimua river prawn',
'African river prawn',
'Giant kelp',
'Giant kelps nei',
'King weakfish',
'Antarctic giant petrel',
'Hall\'s giant petrel',
'Swimming shrimp',
'Knobby swimcrab',
'Snipefishes nei',
'Longspine snipefish',
'Roughhead grenadier',
'Bigeye grenadier',
'Grenadiers nei',
'Ocean pout',
'Cape grenadier',
'Patagonian grenadier',
'Blue grenadier',
'Agate troughshell',
'Californian mactra',
'Rayed trough shell',
'Wedge trough shell',
'Smooth mactra',
'Grey rough shell',
'Largilliert\'s mactra',
'Luzon troughshell',
'Maculated troughshell',
'Plain troughshell',
'Polished mactra',
'Rostrate mactra',
'Trough shells nei',
'Concealed surf clam',
'Globose clam',
'Violet troughshell',
'Panamic winged mactra',
'Mactra surf clams nei',
'Southern barracudina',
'Panamic spidercrab',
'Lesser spider crab',
'Spiny spider crab',
'Maja spider crabs nei',
'Spinous spider crab',
'Black marlin',
'Blue marlin',
'Marlins nei',
'Sand tilefish',
'Diamond back terrapins',
'Krefft\'s ray',
'Smooth skate',
'Soft skate',
'Grinning tun',
'Pacific cask shell',
'Kerguelen malletia',
'White hammer oyster',
'Black hammer oyster',
'Straight hammer oyster',
'Capelin',
'Aquatic mammals nei',
'Antarctic armless flounder',
'Giant manta',
'Cat\'s paw scallop',
'Fertile venus',
'Striate martesia',
'Sevenstar flying squid',
'Megamouth shark',
'Torpedo scad',
'Wuchang bream',
'Large cranch squid',
'Tarpon',
'Indo-Pacific tarpon',
'Norwegian krill',
'Golden callista',
'Squalid callista',
'Humpback whale',
'Great keyhole limpet',
'White-winged scoter',
'Black scoter',
'Haddock',
'Slitted sand dollar',
'Diadem volute',
'Indian volute',
'West Indian crown conch',
'Pacific melongena',
'Moonfish',
'Atlantic silverside',
'Stridulating stone crab',
'Black stone crab',
'Lumpy stone crab',
'Southern kingcroaker',
'Gulf kingcroaker',
'Northern kingfish',
'Kingcroakers nei',
'Northern quahog(=Hard clam)',
'Japanese hard clam',
'Lyrate hard clam',
'Asiatic hard clam',
'Goosander',
'Red-breasted merganser',
'Whiting',
'Merluccid hakes nei',
'Offshore silver hake',
'Panama hake',
'Southern hake',
'Silver hake',
'Shallow-water Cape hake',
'Cape hakes',
'South Pacific hake',
'Argentine hake',
'European hake',
'Deep-water Cape hake',
'Patagonian hake',
'Benguela hake',
'North Pacific hake',
'Senegalese hake',
'Hakes nei',
'Maidenhair mactra',
'Pellucid mactra',
'Macha clam',
'Knout goby',
'Antarctic cranch squid',
'Salmon shrimp',
'Sowerby\'s beaked whale',
'Andrews\' beaked whale',
'Hubbs\' beaked whale',
'Blainville\'s beaked whale',
'Gervais\' beaked whale',
'Ginkgo-toothed beaked whale',
'Gray\'s beaked whale',
'Hector\'s beaked whale',
'Strap-toothed whale',
'True\'s beaked whale',
'Pigmy beaked whale',
'Stejneger\'s beaked whale',
'Nutshell shrimp',
'Andaman lobster',
'Arafura lobster',
'Northwest lobster',
'Caribbean lobster',
'New Zealand lobster',
'Formosa lobster',
'Japanese lobster',
'Mozambique lobster',
'Neptune lobster',
'Urugayian lobster',
'Okinawa sculptured lobster',
'China lobster',
'Metanephrops lobsters nei',
'Red-banded lobster',
'Tora velvet shrimp',
'Rice velvet shrimp',
'Whiskered velvet shrimp',
'Beebe\'s velvet shrimp',
'Reef shrimp',
'Scout velvet shrimp',
'Kishi velvet shrimp',
'Caribbean velvet shrimp',
'Minstrel shrimp',
'Insular velvet shrimp',
'Humpback shrimp',
'Broad velvet shrimp',
'Mining shrimp',
'Mogi velvet shrimp',
'Northern velvet shrimp',
'Southern velvet shrimp',
'Philip velvet shrimp',
'Pink velvet shrimp',
'Fiddler shrimp',
'Tolo velvet shrimp',
'Jinga shrimp',
'Fire shrimp',
'Greentail shrimp',
'Yellow shrimp',
'Wood shrimp',
'Western school shrimp',
'Demons prawn',
'Kadal shrimp',
'York shrimp',
'Fine shrimp',
'Endeavour shrimp',
'Greasyback shrimp',
'Emerald shrimp',
'Middle shrimp',
'Shiba shrimp',
'Ginger shrimp',
'Bird shrimp',
'Eastern school shrimp',
'Speckled shrimp',
'Moyebi shrimp',
'Papua shrimp',
'Metapenaeus shrimps nei',
'Peregrine shrimp',
'Stork shrimp',
'Flamboyant cuttlefish',
'Paintpot cuttlefish',
'Lusitanian sole',
'Frechkop’s sole',
'Thickback soles nei',
'Bastard sole',
'Banded sole',
'Rock violet',
'Grooved sea squirt',
'Sand violet',
'Wormfishes nei',
'Pacific tomcod',
'Atlantic tomcod',
'Southern blue whiting',
'Blue whiting(=Poutassou)',
'Whitemouth croaker',
'Bigeye croaker',
'Croakers nei',
'Atlantic croaker',
'Largemouth black bass',
'Lemon sole',
'Dover sole',
'Box scallop',
'Grey stingfish',
'Northern elephant seal',
'Southern elephant seal',
'Pond loach',
'Weatherfish',
'Harbour spidercrab',
'Channel-clinging crab',
'Horny miter',
'Adusta miter',
'Episcopal miter',
'Brown miter',
'Pontifical miter',
'Zoned miter',
'Mitres nei',
'Goblin shark',
'Lesser devil ray',
'Shortfin devil ray',
'Lesser Guinean devil ray',
'Mantas, devil rays nei',
'Adriatic horse mussel',
'Tulip mussel',
'Furrowed horse mussel',
'Eared horse mussel',
'Bearded horse mussel',
'Capax horse mussel',
'Offshore horse mussel',
'Yellowbanded horse mussel',
'Shiny mussel',
'Philippine horse mussel',
'Straight horse mussel',
'Rhomboid mussel',
'Horse mussels nei',
'False tulip mussel',
'Ocean sunfish',
'Ocean sunfishes nei',
'Freshwater molluscs nei',
'Marine molluscs nei',
'Blue ling',
'Spanish ling',
'Ling',
'Lings nei',
'Filefishes, leatherjackets nei',
'Mediterranean monk seal',
'Hawaiian monk seal',
'Caribbean monk seal',
'Coloured lagoon cockle',
'Full moony',
'African moony',
'Narwhal',
'Articulate monodont',
'Labio monodont',
'Mutable monodont',
'Turbinate monodont',
'Martens’ moonflounder',
'Green laver',
'Humpnose big-eye bream',
'Common mora',
'Chilean lamprey',
'Australian lamprey',
'Moras nei',
'Spaghetti eels nei',
'White perch',
'Striped bass, hybrid',
'Striped bass',
'Northern gannet',
'Cape gannet',
'Australasian gannet',
'Banana mullet',
'Leaping African mullet',
'Flathead grey mullet',
'White mullet',
'Parassi mullet',
'Lebranche mullet',
'So-iuy mullet',
'Mullets nei',
'Taquilla clams',
'Goatfishes, red mullets nei',
'Yellowstripe goatfish',
'Yellow goatfish',
'Orange goatfish',
'Yellowfin goatfish',
'Argentine goatfish',
'Red mullet',
'Surmullets(=Red mullets) nei',
'Surmullet',
'Swarming squat lobster',
'Bristle squat lobster',
'Stout squat lobster',
'Deep-water squat lobster',
'Radiant squat lobster',
'Rugose squat lobster',
'Long-clawed squat lobster',
'Common pike conger',
'Daggertooth pike conger',
'Pike-congers nei',
'Maimed snake eel',
'Morays nei',
'Marbled moray cod',
'Smallhead moray cod',
'Smalleye moray cod',
'Patagonian moray cod',
'Moray cods nei',
'Venus comb murex',
'Murex',
'Black-spined murex',
'Rarespined murex',
'Caltrop murex',
'Murex shells nei',
'Blainville\'s murex',
'Senhouse horse mussel',
'Gummy shark',
'Starry smooth-hound',
'Grey smooth-hound',
'Dusky smooth-hound',
'Sharptooth smooth-hound',
'Striped smooth-hound',
'Spotless smooth-hound',
'Brown smooth-hound',
'Smalleye smooth-hound',
'Spotted estuary smooth-hound',
'Sicklefin smooth-hound',
'Starspotted smooth-hound',
'Speckled smooth-hound',
'Arabian smooth-hound',
'Smooth-hound',
'Narrowfin smooth-hound',
'Whitespotted smooth-hound',
'Blackspotted smooth-hound',
'Narrownose smooth-hound',
'Smooth-hounds nei',
'Humpback smooth-hound',
'Sand gaper',
'Gaper nei',
'Blunt gaper',
'Comb grouper',
'Black grouper',
'Venezuelan grouper',
'Island grouper',
'Yellowmouth grouper',
'Gulf grouper',
'Gag',
'Scamp',
'Leopard grouper',
'Mottled grouper',
'Brazilian groupers nei',
'Tiger grouper',
'Yellowfin grouper',
'Broomtail grouper',
'Lanternfishes nei',
'Eagle rays nei',
'Common eagle ray',
'Bullnose eagle ray',
'Black carp',
'Shorthorn sculpin',
'Sculpins',
'Angel oyster',
'Leopard eel',
'Blotcheye soldierfish',
'Blacktip soldierfish',
'Shoulderbar soldierfish',
'Undulated false lucine',
'Baleen whales nei',
'Guyana swamp mussel',
'Reeve\'s mangrove mussel',
'Strigate mangrove mussel',
'Sea mussels nei',
'Californian mussel',
'Chilean mussel',
'Korean mussel',
'Blue mussel',
'Mediterranean mussel',
'Australian mussel',
'River Plata mussel',
'Mytilus mussels nei',
'Hagfish',
'Hagfishes nei',
'Antarctic limpet',
'Depressed limpet',
'Kerguelen limpet',
'Blackhead salmon',
'Bluespine unicornfish',
'Whitenose shark',
'Francolina jopas',
'Wreath jopas',
'Cake nassa',
'Horned nassa',
'Coronate nassa',
'Burned nassa',
'Cuvier\'s nassa',
'Channeled nassa',
'Gibbous nassa',
'Glans nassa',
'Changeable nassa',
'Natantian decapods nei',
'Flatback turtle',
'Adanson\'s moon snail',
'Broderip\'s moon snail',
'Chemnitz\'s moon snail',
'Collar moon snail',
'Beautifully-banded moon snail',
'Flamed moon snail',
'Gualteri\'s moon snail',
'Lined moon snail',
'Morocco moon snail',
'Starry moon snail',
'Tiger moon snail',
'Turton\'s moon snail',
'Calf moon snail',
'Dillwyn\'s moon-shell',
'Hebrew moon-shell',
'Thousand-spotted moon-shell',
'Ribboned moon-shell',
'Pilotfish',
'Bellybutton nautilus',
'Emperor nautilus',
'Black snake mackerel',
'Small pink lobster',
'Tawny nurse shark',
'Velvet swimcrab',
'Sicklefin lemon shark',
'Lemon shark',
'Sea spaghetti',
'Bloch\'s gizzard shad',
'African spider shrimp',
'Spider shrimp',
'West African estuarine prawn',
'Whitebelly prawn',
'Spider prawn',
'Threadfin and dwarf breams nei',
'Delagoa threadfin bream',
'Japanese threadfin bream',
'Notchedfin threadfin bream',
'Threadfin breams nei',
'Golden threadfin bream',
'Slender threadfin bream',
'Antarctic whelk',
'American sackfish',
'Sackfish',
'Rough king crab',
'Sammara squirrelfish',
'Tropical longfin herring',
'Australian sea lion',
'Finless porpoise',
'Spoon oyster',
'Carol bobtail squid',
'True lobsters,lobsterettes nei',
'Mitten lobsterette',
'Norway lobster',
'Florida lobsterette',
'Prickly lobsterette',
'Scarlet lobsterette',
'Ridge-back lobsterette',
'Pacific lobsterette',
'Two-toned lobsterette',
'Indian Ocean lobsterette',
'Red and white lobsterette',
'Oxpalate nerite',
'Chameleon nerite',
'Costate nerite',
'Bleeding tooth',
'Pitchy nerite',
'Flatspired nerite',
'Plicate nerite',
'Polished nerite',
'Senegal nerite',
'Scaly nerite',
'Turreted nerite',
'Black gemfish',
'Eggwhite moon snail',
'Josephine\'s moon-shell',
'Elephant\'s-foot moon snail',
'Common Atlantic grenadier',
'Marlin-spike grenadier',
'Yellow drum',
'Honnibe croaker',
'Emerald parrotfish',
'Pyramidal prickly-winkle',
'Gambia ark',
'Driftfishes nei',
'Mote sculpin',
'Deep-sea spiny eels nei',
'Spiny-back eel',
'Thomas sea catfish',
'Ringed barracudina',
'Antarctic jonasfish',
'Spotted barracudina',
'Barracudinas nei',
'Broadnose sevengill shark',
'Triangular rockcod',
'Narrowhead rockcod',
'Black rockcod',
'Humped rockcod',
'Striped-eyed rockcod',
'Yellowbelly rockcod',
'Marbled rockcod',
'Notothenia nei',
'Grey rockcod',
'Antarctic rockcods, noties nei',
'Painted notie',
'Toad notie',
'Yellowfin notie',
'Gould\'s flying squid',
'Hawaiian flying squid',
'Wellington flying squid',
'Nototodarus flying squids nei',
'Nuttall\'s mahogany clam',
'Saddled seabream',
'Wilson\'s storm petrel',
'Leach\'s storm-petrel',
'Hedge hog murex',
'Octopuses, etc. nei',
'Ruppell\'s octopus squid',
'Sandbird octopus',
'Caribbean reef octopus',
'Brownstriped octopus',
'Chestnut octopus',
'Big blue octopus',
'Lilliput longarm octopus',
'North Pacific giant octopus',
'Marbled octopus',
'Globe octopus',
'Bumblebee octopus',
'Pygmy octopus',
'Lobed octopus',
'White-spotted octopus',
'Mexican four-eyed octopus',
'Webfoot octopus',
'Spider octopus',
'Moon octopus',
'Octopuses nei',
'Tehuelche octopus',
'Gloomy octopus',
'Whiparm octopus',
'Common octopus',
'Atlantic banded octopus',
'African ghost crab',
'Tufted ghost crab',
'Tuberculate octopus',
'Yellowtail snapper',
'Walrus',
'Smalltooth sand tiger',
'Bigeye sand tiger shark',
'Argentinian silverside',
'Chilean silverside',
'Toothed whales nei',
'Guiana longfin herring',
'Hornlip mullet',
'Telescope shrimp',
'Longjaw leatherjacket',
'Maracaibo leatherjacket',
'Shortjaw leatherjacket',
'Castin leatherjacket',
'Leatherjacket',
'Amethyst olive',
'Purplemouth olive',
'Redmouth olive',
'Common olive',
'Tent olive',
'Netted olice',
'Blood olive',
'Lettered olive',
'Tricolor olive',
'Black olive',
'Neon flying squid',
'Webbed flying squid',
'Flying squids nei',
'Ommastrephidae squids nei',
'Ross seal',
'Golden trout',
'Pink(=Humpback) salmon',
'Chum(=Keta=Dog) salmon',
'Coho(=Silver) salmon',
'Masu(=Cherry) salmon',
'Rainbow trout',
'Sockeye(=Red) salmon',
'Pacific salmons nei',
'Chinook(=Spring=King) salmon',
'Common clubhook squid',
'Boreal clubhook squid',
'Carribean clubhook squid',
'Greater hooked squid',
'Smooth hooked squid',
'Japanese hooked squid',
'Rugose hooked squid',
'Robust clubhook squid',
'Hooked squids nei',
'Snake eels nei',
'Spotted snake eel',
'Punctuated snake-eel',
'Yellow snake-eel',
'Cusk-eels, brotulas nei',
'Band cusk-eel',
'Lingcod',
'Basket, brittle, snake stars',
'Slender thread herring',
'Pacific thread herring',
'Middling thread herring',
'Atlantic thread herring',
'Thread herrings nei',
'Tardoore',
'Irrawaddy dolphin',
'Australian snubfin dolphin',
'Killer whale',
'Spinycheek crayfish',
'Plain bonito',
'Japanese wobbegong',
'Spotted wobbegong',
'Ornate wobbegong',
'Northern wobbegong',
'Tilapias nei',
'Mozambique tilapia',
'Nile tilapia',
'Oreo dories nei',
'Atlantic bird squid',
'Shiny bird squid',
'Pigfish',
'European smelt',
'Rainbow smelt',
'Smelts nei',
'Giant gourami',
'Groundfishes nei',
'Freshwater fishes nei',
'Diadromous fishes nei',
'Marine fishes nei',
'Pelagic fishes nei',
'Finfishes nei',
'Boxfishes nei',
'Chilean flat oyster',
'Denticulate rock oyster',
'European flat oyster',
'Yaquina oyster',
'New Zealand dredge oyster',
'Flat oysters nei',
'Flat and cupped oysters nei',
'Olympia oyster',
'Dwarf oyster',
'South American sea lion',
'Seals nei',
'Tigertooth croaker',
'Common egg cowrie',
'Cheeklined wrasse',
'Prickly dogfish',
'Caribbean roughshark',
'Angular roughshark',
'Sailfin roughshark',
'Frogface goby',
'Punched stone crab',
'Marbled rock crab',
'African matchbox crab',
'Blue hottentot',
'Hottentot seabream',
'Copper breams(=Hottentots) nei',
'Antarctic prion',
'Fairy prion',
'Signal crayfish',
'Axillary seabream',
'Arabian pandora',
'Red pandora',
'Blackspot(=red) seabream',
'Common pandora',
'Natal pandora',
'Pandoras nei',
'Greater snow petrel',
'Lesser snow petrel',
'Snow petrels nei',
'Ivory gull',
'Stocky rockcod',
'Striped rockcod',
'Pagothenia nei',
'Southern common seabream',
'Silver seabream',
'Redbanded seabream',
'Bluespotted seabream',
'Japanese seabream',
'Red porgy',
'Pargo breams nei',
'Baltic prawn',
'Mangrove prawn',
'Rockpool prawn',
'Chinese ditch prawn',
'Tropical river prawn',
'Hilton shrimp',
'Delta prawn',
'Migrant prawn',
'Zaire prawn',
'Caribbean bait prawn',
'Gladiator prawn',
'Indian bait prawn',
'Potitinga prawn',
'Lake prawn',
'Barred grass shrimp',
'Common prawn',
'Carpenter prawn',
'Palaemon shrimps nei',
'Posidonia prawn',
'Pond shrimp',
'Mississippi grass shrimp',
'Eastern grass shrimp',
'Lagoon shrimp',
'Chinese grass shrimp',
'Tonkin grass shrimp',
'Atlantic ditch shrimp',
'Marsh shrimp',
'Ebro shrimp',
'Freshwater prawns, shrimps nei',
'Palaemonid shrimps nei',
'Caribbean furry lobster',
'Indo-Pacific furry lobster',
'Spiny lobsters nei',
'Cape Verde spiny lobster',
'Natal spiny lobster',
'Common spiny lobster',
'Southern spiny lobster',
'Pink spiny lobster',
'Palinurid spiny lobsters nei',
'Buffalo blunthorn lobster',
'American blunthorn lobster',
'Unicorn blunthorn lobster',
'Dulse',
'Silver pomfret',
'Chinese silver pomfret',
'Silver pomfrets nei',
'Pandalid shrimps nei',
'Sidestripe shrimp',
'Morotoge shrimp',
'Pandalopsis shrimps nei',
'Deep-water bigeye shrimp',
'Northern prawn',
'Dock shrimp',
'Humpy shrimp',
'Coonstripe shrimp',
'Ocean shrimp',
'Hokkai shrimp',
'Aesop shrimp',
'Botan shrimp',
'Spot shrimp',
'Pandalus shrimps nei',
'Pacific shrimps nei',
'Striped catfish',
'Aldovrandi\'s panope',
'Pacific geoduck',
'African mud crab',
'Hinged shrimp',
'Caribbean spiny lobster',
'Australian spiny lobster',
'Brown spiny loster',
'Green spiny lobster',
'Spotted spiny lobster',
'Scalloped spiny lobster',
'Blue spiny lobster',
'Mexican spiny loster',
'Japanese spiny lobster',
'Smoothtail spiny lobster',
'Longlegged spiny lobster',
'Banded spiny lobster',
'Ornate spiny lobster',
'Pronghorn spiny lobster',
'Mud spiny lobster',
'Royal spiny lobster',
'Tropical spiny lobsters nei',
'Chinese spiny lobster',
'Painted spiny lobster',
'Rooster venus',
'Semigrooved venus',
'Textile venus',
'Undulate venus',
'White amur bream',
'Japanese snapper',
'Vanuatu snapper',
'Saddle-back snapper',
'Cocoa snapper',
'Yellowtail blue snapper',
'Stony sea urchin',
'Guinean conger',
'Areolated horned shrimp',
'Antarctic escolar',
'Slender escolar',
'Atlantic weasel shark',
'Straight-tooth weasel shark',
'Peruvian rock seabass',
'Barracudinas, etc. nei',
'California flounder',
'Summer flounder',
'Bastard halibut',
'Patagonian flounder',
'Bastard halibuts nei',
'Brown king crab',
'Red king crab',
'Blue king crab',
'King crabs',
'Red stone crab',
'Globose king crab',
'Softshell red crab',
'Antarctic stone crab',
'Red vermillion crab',
'Suco croaker',
'Peruvian banded croaker',
'Broad-mouthed mullet',
'Magellanic rockcod',
'Paranotothenia nei',
'Creole-fish',
'Narwal shrimp',
'Oriental narwal shrimp',
'Hawknose shrimp',
'Arafura shrimp',
'Guinea shrimp',
'Bally shrimp',
'Coral shrimp',
'Coromandel shrimp',
'Spear shrimp',
'Dog shrimp',
'Torpedo shrimp',
'Dwarf shrimp',
'Parole shrimp',
'Rainbow shrimp',
'Parapenaeopsis shrimps nei',
'Kiddi shrimp',
'Smoothshell shrimp',
'Uncta shrimp',
'Adonis shrimp',
'Australian rose shrimp',
'Neptune rose shrimp',
'Explorer rose shrimp',
'Lancer rose shrimp',
'Flamingo shrimp',
'Deep-water rose shrimp',
'Rose shrimp',
'Domino shrimp',
'Parapenaeus shrimps nei',
'Speckled sandperch',
'Doublelined tonguesole',
'Dwarf paradise fish',
'African striped grunt',
'Scalemouth jobfish',
'Smooth dwarf monocle bream',
'Rosy dwarf monocle bream',
'Scaly dwarf monocle bream',
'Collared carpetshark',
'Rusty carpetshark',
'Necklace carpetshark',
'Three spined mantis',
'Oceanian crayfishes nei',
'Black pomfret',
'Nuka shrimp',
'Antarctic octopuses',
'Turquet\'s octopus',
'African sailfin flyingfish',
'Velvet leatherjacket',
'Campeche catshark',
'Blackgill catshark',
'Salamander shark',
'Filetail catshark',
'Paromola',
'Parona leatherjacket',
'Sculptured mitten lobster',
'Caledonian mitten lobster',
'Red-spotted mitten lobster',
'Marbled mitten lobster',
'Dash-and-dot goatfish',
'Cinnabar goatfish',
'Indian goatfish',
'Rosy goatfish',
'Japanese glass shrimp',
'Pink glass shrimp',
'White glass shrimp',
'Patagonian rockcod',
'Longtail Southern cod',
'Patagonotothen nei',
'Rayed mediterranean limpet',
'Ferreous limpet',
'Star-shaped limpet',
'Giant Mexican limpet',
'Black limpet',
'Rustic limpet',
'Limpets nei',
'Rough limpet',
'Limpet',
'Pacific sugar limpet',
'Striate limpet',
'Weathervane scallop',
'Yesso scallop',
'Shoulderblade coral',
'Pillar coral',
'Fungus coral',
'Entangled moon-shell',
'Lunar Mexican scallop',
'San Diego scallop',
'Southern Australia scallop',
'Great Mediterranean scallop',
'Great Atlantic scallop',
'New Zealand scallop',
'Silken scallop',
'Pecten scallops nei',
'Vogde\'s scallop',
'Zigzag scallop',
'Scallops nei',
'Cadenat\'s sole',
'Cyclope sole',
'White-faced storm petrel',
'Fourlined terapon',
'South Georgia diving petrel',
'Indian pellona',
'Yellowfin river pellona',
'Smalltoothed pellonula',
'Penaeid shrimps nei',
'Scythe shrimp',
'Needle shrimp',
'Megalops shrimp',
'Northern brown shrimp',
'Redspotted shrimp',
'Crystal shrimp',
'Yellowleg shrimp',
'Witch prawn',
'Fleshy prawn',
'Northern pink shrimp',
'Brown tiger prawn',
'Indian white prawn',
'Kuruma prawn',
'Caramote prawn',
'Western king prawn',
'Red-spot king prawn',
'Aloha prawn',
'Banana prawn',
'Giant tiger prawn',
'Southern pink shrimp',
'Western white shrimp',
'Sao Paulo shrimp',
'Redtail prawn',
'Eastern king prawn',
'Southern white shrimp',
'Green tiger prawn',
'Northern white shrimp',
'Penaeus shrimps nei',
'Blue shrimp',
'Southern brown shrimp',
'Whiteleg shrimp',
'Donkey croaker',
'Silver croaker',
'Onefin catshark',
'Royal threadfin',
'Melon-headed whale',
'Pacific harvestfish',
'American harvestfish',
'Pacific pompano',
'Salema butterfish',
'Gulf butterfishes, etc. nei',
'Atlantic butterfish',
'American yellow perch',
'European perch',
'Percarina',
'Demersal percomorphs nei',
'Pelagic percomorphs nei',
'Percoids nei',
'Brazilian flathead',
'Clathrate venus',
'Many-ridged venus',
'Youthful venus',
'Reticulated venus',
'Atlantic mudskipper',
'Subantarctic struthiolaria',
'New Zealand mussel',
'South American rock mussel',
'African mussel',
'Perna mussels nei',
'Green mussel',
'Rossignol’s toadfish',
'Brown leafweed',
'False angel wing',
'Big-claw purple hermit crab',
'Sea lamprey',
'Lampreys nei',
'Red steenbras',
'Brown seaweeds',
'Cormorants nei',
'European shag',
'Great cormorant',
'Banded bonnet',
'Pacific Scotch bonnet',
'Grey bonnet',
'Grooved helmet',
'Saburon helmet',
'Sharp razor clam',
'Javanese razor clam',
'Bean solen',
'Caspian seal',
'Ribbon seal',
'Harp seal',
'Ringed seal',
'Larga seal',
'Baikal seal',
'Harbour seal',
'Spectacled porpoise',
'New Zealand sea lion',
'Harbour porpoise',
'Vaquita',
'Burmeister\'s porpoise',
'Dall\'s porpoise',
'Short-tailed albatross',
'Laysan albatross',
'Waved albatross',
'Black-footed albatross',
'Sooty albatross',
'Light-mantled sooty albatross',
'Pacific coast angel wing',
'Common piddock',
'Oriental angel wing',
'Scaled squid',
'Coffeebean scaled squid',
'Greater forkbeard',
'Longfin hake',
'Forkbeard',
'Forkbeards nei',
'Cabbage murex',
'Duplex murex',
'Pink-mouthed murex',
'Margarita murex',
'Apple murex',
'Regal murex',
'Banded murex',
'Membraneous phyllophora',
'Phyllophora nei',
'Chalk weed',
'Sperm whale',
'Japanese pearl oyster',
'Spotted pearl oyster',
'Blacklip pearl oyster',
'Silverlip pearl oyster',
'Mazatlan pearl oyster',
'Rayed pearl oyster',
'Pearl oysters nei',
'Brazilian sandperch',
'Chilean sandperch',
'Pinjalo',
'Bicolor pen shell',
'Prickly pen shell',
'Noble pen shell',
'Rough pen shell',
'Rugose pen shell',
'Striate pisania',
'Rice-paddy eel',
'Ridged pitar venus',
'Yellow pitar venus',
'Pellucid pitar venus',
'Rough pitar venus',
'Swollen venus',
'Self-coloured pitar',
'Tiar venus',
'American sea scallop',
'Saddle oyster',
'Windowpane oyster',
'Broad cockle',
'South American silver croaker',
'Aquatic plants nei',
'Ganges River dolphin',
'Indus River dolphin',
'Batfishes',
'European flounder',
'Starry flounder',
'Keeltail needlefish',
'Flatheads nei',
'Bartail flathead',
'Paco box crab',
'Ayu sweetfish',
'Dusky rubberlip',
'Blackspotted rubberlip',
'Harry hotlips',
'Biglip grunt',
'Rubberlip grunt',
'Trout sweetlips',
'Whitebarred rubberlip',
'Minstrel sweetlips',
'Sordid rubberlip',
'Squaretail coralgrouper',
'Blacksaddled coralgrouper',
'Leopard coralgrouper',
'Spotted coralgrouper',
'Highfin coralgrouper',
'Roving coralgrouper',
'Marbled coralgrouper',
'Argentine red shrimp',
'Royal red shrimp',
'Pleoticus shrimps nei',
'Lesser striped shrimp',
'Gondwana striped shrimp',
'Catalonian striped shrimp',
'Scarlet longbeak shrimp',
'Giant longbeak shrimp',
'Striped soldier shrimp',
'Striped gladiator shrimp',
'Italian deep-sea shrimp',
'Arrow shrimp',
'Golden shrimp',
'Mexican longbeak shrimp',
'Plesionika shrimps nei',
'Colombian longbeak shrimp',
'Guinea striped shrimp',
'Scarlet shrimp',
'Antarctic silverfish',
'Okhotsk atka mackerel',
'Atka mackerel',
'Carrot squat lobster',
'Pelagic red crab',
'European plaice',
'Alaska plaice',
'English sole',
'Righteye flounders nei',
'Flatfishes nei',
'Prince horse conch',
'Salmon horse conch',
'Trapezium horse conch',
'Thinspine sea catfish',
'Yellowfin herring',
'Sixgill sawshark',
'Plocamium',
'Eeltail catfishes',
'Cauliflower coral',
'Elkhorn cora',
'Brocoli coral',
'Rasp coral',
'Horned grebe',
'Great crested grebe',
'Red-necked grebe',
'Arrau river turtle',
'Abalone jingle shell',
'Black drum',
'Bladder moon snail',
'Pear-shaped moon snail',
'Blackmouth moon snail',
'Seba\'s moon snail',
'Pollack',
'Saithe(=Pollock)',
'Pacific goose barnacle',
'Barnacle',
'Leaf barnacle',
'Knife-back seabream',
'Henslow’s swimming crab',
'Marine worms',
'Blue bobo',
'Slender fivefinger threadfin',
'Long-limb threadfin',
'King threadfin',
'River threadfin',
'African blackspot threadfin',
'Smallmouth threadfin',
'Arabian blackspot threadfin',
'Australian threadfin',
'Blackfin threadfin',
'Atlantic threadfin',
'Littlescale threadfin',
'Yellow bobo',
'Persian blackspot threadfin',
'Striped threadfin',
'Giant African threadfin',
'Sixfinger threadfin',
'Blackspot threadfin',
'Largemouth striped threadfin',
'Barbu',
'Equilateral marsh clam',
'Slender marsh clam',
'Bengali geloina',
'Common geloina',
'Broad geloina',
'Inflated marsh clam',
'Stout beardfish',
'Threadfins, tasselfishes nei',
'Northern paradise fish',
'Eastern paradise fish',
'Hornaday’s paradise fish',
'Kapuas elegant paradise fish',
'Blackhand paradise fish',
'Elegant paradise fish',
'Paradise threadfin',
'Mississippi paddlefish',
'Wreckfish',
'Hapuku wreckfish',
'Frenchman seabream',
'Blueskin seabream',
'Scotsman seabream',
'Polystegan seabreams nei',
'Seventyfour seabream',
'Steller\'s eider',
'Noble giant turrid',
'White keeled turrid',
'Angelfishes nei',
'Semicircle angelfish',
'Damselfishes',
'Silver grunt',
'Bluecheek silver grunt',
'Banded grunter',
'Bastard grunt',
'Sompat grunt',
'Javelin grunter',
'Panama grunt',
'Parrot grunt',
'Pigsnout grunt',
'Striped piggy',
'Bluefish',
'Common goby',
'Sand goby',
'White crappie',
'Offshore rockfish',
'Speckled deepwat. scorpionfish',
'Speckled scorpionfish',
'Hardshell shrimp',
'Feather shrimp',
'Spiny shrimp',
'Franciscana',
'Daisy midshipman',
'Smooth star coral',
'Small star coral',
'Striped catshark',
'Barbeled catshark',
'Leopard catshark',
'Spotted nori',
'Ribboned nori',
'Frilled nori',
'Nori nei',
'Laver (Nori)',
'Kerguelen nut shell',
'Slender swimcrab',
'Swimming crabs, etc. nei',
'Rough swimcrab',
'Iridescent swimming crab',
'Lancer swimcrab',
'Blue swimming crab',
'Blotched swimming crab',
'Portunus swimcrabs nei',
'Gazami crab',
'Senegalese smooth swimcrab',
'Xantus swimcrab',
'Neptune-grass',
'Bigeyes,glasseyes,bulleyes nei',
'Paeony bulleye',
'Moontail bullseye',
'Red bigeye',
'Bigeyes nei',
'Blue shark',
'Atlantic searobins',
'Sawfishes',
'Longnose sawshark',
'Japanese sawshark',
'Shortnose sawshark',
'Bahamas sawshark',
'Sawsharks nei',
'Wenchman',
'Ornate jobfish',
'Goldflag jobfish',
'Crimson jobfish',
'Golden eye jobfish',
'Cardinal snapper',
'Goldbanded jobfish',
'Lavender jobfish',
'Jobfishes nei',
'Sharptooth jobfish',
'Oblique-banded snapper',
'Red swamp crawfish',
'Procambarus crayfishes nei',
'White-chinned petrel',
'Grey petrel',
'Spectacled petrel',
'Parkinson\'s petrel',
'Westland petrel',
'Processa shrimp',
'Nika shrimp',
'Peruvian one-handed shrimp',
'Processa shrimps nei',
'Streaked prochilod',
'Netted prochilod',
'Prochilods nei',
'Bluntnose flyingfish',
'Chilean jagged lobster',
'Pacific sandperch',
'Roudi escolar',
'Graceful catshark',
'African lungfishes',
'Rough littleneck',
'Columbian littleneck',
'Lesser littleneck',
'Pacific littleneck clam',
'Thin-shelled littleneck',
'Taca clam',
'Titi shrimp',
'Challenger volute',
'Green sea urchin',
'Pacific rudderfish',
'Turbot',
'Pacific sand sole',
'Spottail spiny turbot',
'Spiny turbot',
'Indian halibut',
'Spiny turbots nei',
'Club scallop',
'Stone triggerfish',
'Juan Fernandez trevally',
'White trevally',
'Crocodile shark',
'Imperial surf clam',
'South Georgia icefish',
'Pelagic armourhead',
'Pelagic armourheads nei',
'Red codling',
'Winter flounder',
'Yellow striped flounder',
'False killer whale',
'Cinnamon flounder',
'Malayan flounder',
'Three spotted flounder',
'Bobo croaker',
'Guinea croaker',
'Cassava croaker',
'Law croaker',
'West African croakers nei',
'Longneck croaker',
'False catshark',
'Aluco vergatus',
'West African goatfish',
'Glacial squid',
'Swift wing oyster',
'European wing oyster',
'Penguin wing oyster',
'Western wing oyster',
'Capricorn fusilier',
'Goldband fusilier',
'Double-lined fusilier',
'Banana fusilier',
'One-stripe fusilier',
'Dark-banded fusilier',
'Spanish agar',
'Fourhorn octopus',
'Trindade petrel',
'Kerguelen petrel',
'Bermuda petrel',
'White-necked petrel',
'Black-capped petrel',
'Mottled petrel',
'White-headed petrel',
'Gould\'s Petrel',
'Great-winged petrel',
'Providence petrel',
'Panga seabream',
'Devil firefish',
'Boe drum',
'Longfin bonefish',
'Roundear enope squid',
'Latchet(=Sharpbeak gurnard)',
'Magellanic ptilonia',
'Banded whip lobster',
'Red whip lobster',
'Arabian whip lobster',
'Whip lobsters nei',
'Velvet whip lobster',
'Flesh-footed shearwater',
'Pink-footed shearwater',
'Fluttering shearwater',
'Great shearwater',
'Sooty shearwater',
'Huttons shearwater',
'Audubon\'s shearwater',
'Balearic shearwater',
'Wedge-tailed shearwater',
'Manx shearwater',
'Short-tailed shearwater',
'Yelkouan shearwater',
'Spiral melongena',
'Colossal melongena',
'Giant hairy melongena',
'Ternate melongena',
'Asian barbs nei',
'Rudolph\'s purpura',
'Wide-foot purpura',
'Persian purpura',
'Ebony miter',
'Adelie penguin',
'Chinstrap penguin',
'Gentoo penguin',
'Dotted dove shell',
'Jewel enope squid',
'Common pythia',
'Red sea squirt',
'Cobia',
'White skate',
'Mediterranean starry ray',
'Blue skate',
'Blonde ray',
'Sandy ray',
'Thornback ray',
'Little skate',
'Shagreen ray',
'Round ray',
'Antarctic starry skate',
'Arctic skate',
'Shorttail skate',
'Barndoor skate',
'Sailray',
'Madeiran ray',
'Small-eyed ray',
'Brown ray',
'Spotted ray',
'Cuckoo ray',
'Norwegian skate',
'Winter skate',
'Longnosed skate',
'Starry ray',
'Raja rays nei',
'Undulate ray',
'Bigthorn skate',
'Rays and skates nei',
'Rays, stingrays, mantas nei',
'Jenyns\'s sprat',
'American bull frog',
'Frogs',
'Randall\'s snapper',
'Common rangia',
'Western Mexican rangia',
'Tadpole fish',
'Turnish shaped rapa',
'Sea snails',
'Thomas\' rapa whelk',
'Short mackerel',
'Island mackerel',
'Indian mackerel',
'Indian mackerels nei',
'Oarfishes nei',
'King of herrings',
'Greenland halibut',
'Spearfish remora',
'Marlin sucker',
'Shark sucker',
'Lobsters nei',
'Long-finned escolar',
'Bengal escolar',
'Short-lined escolar',
'Nakamura\'s escolar',
'Royal escolar',
'Silver gemfish',
'Paxton\'s escolar',
'Swallowtail cardinalfish',
'White stumpnose',
'Haffara seabream',
'Goldlined seabream',
'Whale shark',
'Guitarfishes, etc. nei',
'Whitespotted guitarfish',
'Chola guitarfish',
'Pacific guitarfish',
'Guitarfishes nei',
'Straightnose rabbitfish',
'Knife-nosed chimaeras nei',
'Rough vergatus',
'Banded vertagus',
'Obelisk vergatus',
'Common vergatus',
'Humboldt\'s potamid',
'Milk shark',
'Brazilian sharpnose shark',
'Pacific sharpnose shark',
'Grey sharpnose shark',
'Caribbean sharpnose shark',
'Sharpnose sharks nei',
'Australian sharpnose shark',
'Atlantic sharpnose shark',
'Red seaweeds',
'Vermilion snapper',
'Sand flounders nei',
'Jellyfishes nei',
'Giant guitarfish',
'Rabbitnose shrimp',
'Long billed halfbeak',
'Black-legged kittiwake',
'Rock chickory',
'Thorny flathead',
'Lentil bobtail squid',
'Bully bobtail squid',
'Stout bobtail squid',
'North Pacific bobtail squid',
'Tortuga bobtail squid',
'Grooved carpet shell',
'Japanese carpet shell',
'Carpet shells nei',
'Variegated carpet shell',
'Roach',
'Roaches nei',
'Oilfish',
'Spotted soapfish',
'Sydney cupped oyster',
'Hooded oyster',
'Palmate oyster',
'Tadpole codling',
'Atlantic salmon',
'Trouts nei',
'Sea trout',
'Salmonoids nei',
'Golden grouper',
'Salps',
'Arctic char',
'Brook trout',
'Dolly varden',
'Lake trout(=Char)',
'Chars nei',
'Melancholy cranch squid',
'Sauger',
'Pike-perch',
'Walleyes nei',
'Walleye',
'Australian bonito',
'Eastern Pacific bonito',
'Striped bonito',
'Atlantic bonito',
'Bonitos nei',
'European pilchard(=Sardine)',
'White sardinella',
'Round sardinella',
'Brazilian sardinella',
'Fringescale sardinella',
'Goldstripe sardinella',
'Bali sardinella',
'Indian oil sardine',
'Madeiran sardinella',
'Blacktip sardinella',
'Yellowtail sardinella',
'Sind sardinella',
'Sardinellas nei',
'Japanese sardinella',
'California pilchard',
'Japanese pilchard',
'Southern African pilchard',
'South American pilchard',
'Pilchards nei',
'Common sargassum',
'Red squirrelfish',
'Redcoat',
'Sabre squirrelfish',
'Salema',
'Brazilian lizardfish',
'Gracile lizardfish',
'Longfin lizardfish',
'Clouded lizardfish',
'Greater lizardfish',
'Brushtooth lizardfish',
'Butter clam',
'Unihorn octopus',
'Scalpellidae barnacles nei',
'Sowerby\'s ark',
'Incongruous ark',
'Corneous ark',
'Globose ark',
'Inequivalve ark',
'Rudder ark',
'Pill ark',
'Half-crenated ark',
'Rudd',
'Parrotfishes nei',
'Arabian parrotfish',
'Rusty parrotfish',
'Bridled parrotfish',
'Purple-brown parrotfish',
'Blue-barred parrotfish',
'Guinean parrotfish',
'Dusky parrotfish',
'Gulf parrotfish',
'Common parrotfish',
'Ember parrotfish',
'Eclipse parrotfish',
'Tricolour parrotfish',
'Scats',
'Cornish blackfish',
'Imperial blackfish',
'Pemarco blackfish',
'Schedophilus nei',
'Narrowmouthed catshark',
'Redspotted catshark',
'Narrowtail catshark',
'Slender catshark',
'Couma sea catfish',
'Brown sea catfish',
'Pemecou sea catfish',
'Passany sea catfish',
'Sciaenas nei',
'Brown meagre',
'Croakers, drums nei',
'Red drum',
'Tule nei',
'Hard corals, madrepores nei',
'Four-spines nutshell shrimp',
'Bering shrimp',
'Sculptured shrimps nei',
'Asian bonytongue',
'Spadenose shark',
'Yellowstripe monocle bream',
'Two-lined monocle bream',
'Thumbprint monocle bream',
'Bridled monocle bream',
'Arabian monocle bream',
'Striped monocle bream',
'Monocle breams',
'Black-streaked monocle bream',
'Whitecheek monocle bream',
'Blue mackerel',
'Chub mackerel',
'Atlantic mackerel',
'Scomber mackerels nei',
'Sauries nei',
'Atlantic saury',
'Dwarf saury',
'Talang queenfish',
'Doublespotted queenfish',
'Queenfishes',
'Barred queenfish',
'Needlescaled queenfish',
'Serra Spanish mackerel',
'King mackerel',
'Narrow-barred Spanish mackerel',
'Monterey Spanish mackerel',
'Indo-Pacific king mackerel',
'Korean seerfish',
'Streaked seerfish',
'Atlantic Spanish mackerel',
'Papuan seerfish',
'Australian spotted mackerel',
'Japanese Spanish mackerel',
'Kanadi kingfish',
'Queensland school mackerel',
'Cero',
'Broad-barred king mackerel',
'Pacific sierra',
'Chinese seerfish',
'Seerfishes nei',
'West African Spanish mackerel',
'Mackerels nei',
'Tuna-like fishes nei',
'Turbots nei',
'Windowpane flounder',
'Brill',
'Slender rockfish',
'Madeira rockfish',
'Black scorpionfish',
'Red scorpionfish',
'Scorpionfishes, rockfishes nei',
'Spotted-fin rockfish',
'Scorpionfishes nei',
'Raggy scorpionfish',
'Peppery furrow',
'Catsharks, etc. nei',
'Polkadot catshark',
'Boa catshark',
'Small-spotted catshark',
'Yellowspotted catshark',
'West African catshark',
'Brownspotted catshark',
'Freckled catshark',
'Whitesaddled catshark',
'Blotched catshark',
'Chain catshark',
'Catsharks, nursehounds nei',
'Nursehound',
'Cloudy catshark',
'Dwarf catshark',
'Indo-Pacific swamp crab',
'Slipper lobsters nei',
'Spanish slipper lobster',
'Galapagos slipper lobster',
'Brazilian slipper lobster',
'Hooded slipper lobster',
'Three-spot slipper lobster',
'Cape slipper lobster',
'Red slipper lobster',
'Mediterranean slipper lobster',
'Ridged slipper lobster',
'Blunt slipper lobster',
'Lesser slipper lobster',
'Soft locust lobster',
'Two-spotted locust lobster',
'Blue-back locust lobster',
'Striated locust lobster',
'Pygmy locust lobster',
'Locust lobsters nei',
'Flapnose houndshark',
'Sherwood dogfish',
'Smallmouth knifetooth dogfish',
'Knifetooth dogfish',
'Scymnodon dogfishes nei',
'Pacific ocean perch',
'Cape redfish',
'Widow rockfish',
'Acadian redfish',
'Yellowtail rockfish',
'Chilipepper rockfish',
'Golden redfish',
'Beaked redfish',
'Bocaccio rockfish',
'Canary rockfish',
'Atlantic redfishes nei',
'Norway redfish',
'Various sharks nei',
'Oxeye scad',
'Bigeye scad',
'Yellowstripe scad',
'Hairfin lookdown',
'African moonfish',
'Peruvian moonfish',
'Atlantic moonfish',
'Lookdown',
'Bark semele',
'Chilean semele',
'Semeles nei',
'California sheephead',
'Greater shining bobtail squid',
'Lesser shining bobtail squid',
'Heavy African ark',
'Needle cuttlefish',
'Andrea cuttlefish',
'Giant Australian cuttlefish',
'Arabian cuttlefish',
'Southern cuttlefish',
'Stumpy cuttlefish',
'African cuttlefish',
'Slender cuttlefish',
'Shortclub cuttlefish',
'Knifebone cuttlefish',
'Elegant cuttlefish',
'Ovalbone cuttlefish',
'Guinean cuttlefish',
'Golden cuttlefish',
'Ken’s cuttlefish',
'Hedley’s cuttlefish',
'Giant African cuttlefish',
'Kobi cuttlefish',
'Broadclub cuttlefish',
'Longarm cuttlefish',
'Spider cuttlefish',
'Kisslip cuttlefish',
'Madokai\'s cuttlefish',
'Reaper cuttlefish',
'Frog cuttlefish',
'Common cuttlefish',
'Oman cuttlefish',
'Magnificent cuttlefish',
'Pink cuttlefish',
'Papuan cuttlefish',
'Pharaoh cuttlefish',
'Striking cuttlefish',
'Small striped cuttlefish',
'Hooded cuttlefish',
'Large striped cuttlefish',
'Curvespine cuttlefish',
'Rosecone cuttlefish',
'Broadback cuttlefish',
'Smith’s cuttlefish',
'Cuttlefishes nei',
'Starry cuttlefish',
'Grooved cuttlefish',
'Trident cuttlefish',
'Patchwork cuttlefish',
'Viet Nam cuttlefish',
'Voss’ cuttlefish',
'Whitley’s cuttlefish',
'Zanzibar cuttlefish',
'Southern bottletail squid',
'Koch’s bottletail squid',
'Spineless cuttlefish',
'Japanese spineless cuttlefish',
'Ornate cuttlefish',
'Sepiella cuttlefishes nei',
'Web’s cuttlefish',
'Elegant bobtail squid',
'Mysterious bobtail squid',
'Common bobtail squid',
'Cuttlefish, bobtail squids nei',
'Analogous bobtail squid',
'Atlantic bobtail squid',
'Golden bobtail squid',
'Butterfly bobtail squid',
'Intermediate bobtail squid',
'Tongue bobtail squid',
'Spotty bobtail squid',
'Robust bobtail squid',
'Dwarf bobtail squid',
'Sepiola bobtail squids nei',
'Steenstrup\'s bobtail squid',
'Knobby bobtail squid',
'Japanese bobtail squid',
'Southern reef squid',
'Bigfin reef squid',
'Caribbean reef squid',
'Reef squids nei',
'Box mussel',
'Sakura shrimp',
'Sergestid shrimps nei',
'Guinean amberjack',
'Greater amberjack',
'Lesser amberjack',
'Yellowtail amberjack',
'Fortune jack',
'Japanese amberjack',
'Longfin yellowtail',
'Amberjacks nei',
'Banded rudderfish',
'Choicy ruff',
'Warehou nei',
'Palm ruff',
'Blackbanded trevally',
'Snake-like worm shell',
'Jellyfish worm shell',
'Groupers, seabasses nei',
'Ghanian comber',
'Blacktail comber',
'Comber',
'Combers nei',
'Piranhas nei',
'Greenland smoothcockle',
'Angola marsh crab',
'Hairy matchbox crab',
'Coloured rock shrimp',
'Hardhusk rock shrimp',
'Rock shrimp',
'Burkenroad\'s rock shrimp',
'Mediterranean rock shrimp',
'Ridgeback rock shrimp',
'Keeled rock shrimp',
'Target shrimp',
'Notched rock shrimp',
'Lesser rock shrimp',
'Tufted rock shrimp',
'Pacific rock shrimp',
'Notched tidal rock shrimp',
'Knight rock shrimp',
'Martin\'s rock shrimp',
'Mixed rock shrimp',
'Peanut rock shrimp',
'Small peanut rock',
'Eyespot rock shrimp',
'Kinglet rock shrimp',
'Bichique',
'White-spotted spinefoot',
'Goldlined spinefoot',
'Marbled spinefoot',
'Spinefeet(=Rabbitfishes) nei',
'Pacific razor clam',
'Winter\'s razor clam',
'Sillago-whitings',
'Spotted sillago',
'Flathead sillago',
'Arabian sillago',
'Slender sillago',
'Clubfoot sillago',
'Sand sillago',
'Flinders\' sillago',
'Trumpeter sillago',
'Silver sillago',
'Freshwater siluroids nei',
'Wels(=Som) catfish',
'Concave baby\'s ear',
'Giant false limpet',
'Javanese false limpet',
'Fringed false limpet',
'Sirius false limpet',
'Egyptian sole',
'Elongate sole',
'Sand sole',
'Senegalese sole',
'Common sole',
'Divaricate short razor',
'Rasp short razor',
'Soles nei',
'Mediterranean awning clam',
'Cape razor clam',
'Grand razor shell',
'Guinea razor shell',
'Lamarck\'s razor shell',
'European razor clam',
'Pink-spotted razor shell',
'Rostrate jackknife',
'Rough jackknife',
'Solen razor clams nei',
'Razor clams, knife clams nei',
'African mud shrimp',
'Kolibri shrimp',
'Ridgeback shrimp',
'Coastal mud shrimp',
'Flower shrimp',
'Guiana mud shrimp',
'Deep-sea mud shrimp',
'Chinese mud shrimp',
'Atlantic mud shrimp',
'Blossom shrimp',
'Comb shrimp',
'Solenocera shrimps nei',
'Solenocerid shrimps nei',
'Gates\' goblet',
'Diphos sanguin',
'False sea pine',
'Guinea flathead',
'Common eider',
'King eider',
'Greenland shark',
'Pacific sleeper shark',
'Little sleeper shark',
'Tuberculated flathead',
'Tucuxi',
'Guyana dolphin',
'Indo-Pac. hump-backed dolphin',
'Atlantic hump-backed dolphin',
'Porgies, seabreams nei',
'Sobaity seabream',
'Parrotfish',
'Redfin parrotfish',
'Gilthead seabream',
'Violet sea urchin',
'False gorgon',
'Penguins nei',
'Northern puffer',
'Bandtail puffer',
'Atlantic puffers nei',
'Guinean barracuda',
'Great barracuda',
'Mexican barracuda',
'Bigeye barracuda',
'Pickhandle barracuda',
'Obtuse barracuda',
'European barracuda',
'Barracudas nei',
'Yellowmouth barracuda',
'Barracudas, etc. nei',
'Scalloped bonnethead',
'Whitefin hammerhead',
'Scalloped hammerhead',
'Scoophead',
'Great hammerhead',
'Hammerhead sharks nei',
'Bonnethead',
'Smalleye hammerhead',
'Smooth hammerhead',
'Hammerhead sharks, etc. nei',
'Bigeye picarel',
'Blotched picarel',
'Blackspot picarel',
'Blacktail picarel',
'Picarel',
'Picarels nei',
'Friendly spine shrimp',
'Parrot shrimp',
'Ram\'s horn squid',
'Spirulina nei',
'Sea spirulina',
'Oval surf clam',
'Stimpson\'s surf clam',
'Solid surf clam',
'Atlantic surf clam',
'Surf clams nei',
'Subtruncate surf clam',
'Black seabream',
'Steentjie seabream',
'Bearded thorny oyster',
'Butler\'s thorny oyster',
'Donkey thorny oyster',
'European thorny oyster',
'Imperial thorny oyster',
'Pacific thorny oyster',
'Thorny oysters nei',
'Ducal thorny oyster',
'Golden thorny oyster',
'Elephant ear',
'Shiny sponge',
'Greek bathing sponge',
'Leather sponge',
'Sponges',
'Delicate round herring',
'Silver-stripe round herring',
'Falkland sprat',
'European sprat',
'Dogfish sharks nei',
'Dogfishes and hounds nei',
'Dogfish sharks, etc. nei',
'Spined pygmy shark',
'Picked dogfish',
'Longnose spurdog',
'Cuban dogfish',
'Japanese spurdog',
'Shortnose spurdog',
'Blacktailed spurdog',
'Shortspine spurdog',
'Cyrano spurdog',
'Dogfishes nei',
'Sawback angelshark',
'African angelshark',
'Argentine angelshark',
'Australian angelshark',
'Pacific angelshark',
'Sand devil',
'Taiwan angleshark',
'Japanese angelshark',
'Clouded angelshark',
'Smoothback angelshark',
'Angelshark',
'Ornate angelshark',
'Ocellated angelshark',
'Angelsharks, sand devils nei',
'Kicking mantis shrimp',
'Sorcerer mantis shrimp',
'Bigelow mantis shrimp',
'Angolan mantis srimp',
'Rough mantis shrimp',
'Spottail mantis squillid',
'Catalina mantis',
'Panama mantis shrimp',
'Small mantis shrimp',
'Squillids nei',
'Distorted mussel',
'Zebra shark',
'Luminous hake',
'Hollow stardrum',
'White stardrum',
'Pantropical spotted dolphin',
'Clymene dolphin',
'Striped dolphin',
'Atlantic spotted dolphin',
'Spinner dolphin',
'Spotted dolphins nei',
'Rough-toothed dolphin',
'Velvet spidercrab',
'Scup',
'Reticulated leatherjacket',
'Arctic skua',
'Skua',
'Giant seabass',
'Sooty tern',
'Common tern',
'Arctic tern',
'Antarctic tern',
'Hatchetfishes nei',
'Purpleback flying squid',
'Orangeback flying squid',
'Greenfish',
'Royal cucumber',
'Curryfish',
'Commerson\'s anchovy',
'Indian anchovy',
'Stolephorus anchovies nei',
'Spotty-face anchovy',
'Leucoptera bobtail squid',
'Dagaas (=Kapenta)',
'Stomatopods nei',
'Cannonball jellyfish',
'Araucanian herring',
'Stone oyster',
'Butterfishes, pomfrets nei',
'Blue butterfish',
'Butterfishes nei',
'Starry butterfish',
'Slender strombina',
'Stromboli keyhole limpet',
'Diana conch',
'Bubble conch',
'Dog conch',
'Milk conch',
'Samar conch',
'Swan conch',
'Giant Eastern Pacific conch',
'Roster-tail conch',
'Gibbose conch',
'Queen conch',
'Eastern Pacific fighting conch',
'Granulated conch',
'Plicate conch',
'Widest Pacific conch',
'West African stromb',
'Silver conch',
'Strawberry conch',
'Marginate conch',
'Mutable conch',
'Cock\'s comb conch',
'Fighting conch',
'Hawk-wing conch',
'Laciniate conch',
'Stromboid conchs nei',
'Three-knobbed conch',
'Little pitcher conch',
'Variable conch',
'Sea urchins nei',
'Atlantic needlefish',
'Senegal needlefish',
'Sea potato',
'Hydrocorals',
'Masked triggerfish',
'Mauve sunetta',
'Truncate sunetta',
'Cobbler wobbegong',
'Papillose flounder',
'Dusky flounder',
'Large-scale lantern fish',
'Grey wrasse',
'Corkwing wrasse',
'Symphodus wrasses nei',
'Sailfin snapper',
'Chinamanfish',
'Vanmelle’s tonguefish',
'Luminous flying squid',
'Aden splitfin',
'Blackmouth splitfin',
'Splitfins nei',
'Stonefish',
'Guinean sole',
'Commerson\'s sole',
'Klein\'s sole',
'Phaeton dragonet',
'Pipefishes, seahorses nei',
'Lizardfishes nei',
'Sand lizardfish',
'Triplecross lizardfish',
'Snakefish',
'Redbarred lizardfish',
'Australian trumpet',
'Common shelduck',
'Adanson\'s tagelus',
'Californian tagelus',
'Dombey\'s tagelus',
'Duckbill tagelus',
'Purple pufferfish',
'Talona pholaad',
'Dana octopus squid',
'Peacock cranch squid',
'Belcher\'s venus',
'Turgid venus',
'Lettered venus',
'Sherpherd\'s beaked whale',
'Tautog',
'Cunner',
'Coronate prickly-winkle',
'Hailstorm prickly-winkle',
'Pagoda prickly-winkle',
'Flamed moon-shell',
'Dentate top shell',
'Fenestrate top',
'Pyramid top',
'Snakeskin tegula',
'Telescope snail',
'Glossy tellin',
'Foliated tellin',
'Hyaline tellin',
'Fleshy tellin',
'Lacerate tellin',
'Cat\'s tongue tellin',
'Palate tellin',
'Flat tellin',
'Beautiful tellin',
'Remies tellin',
'Rasp tellin',
'Senegambian tellin',
'Similar red tellin',
'Tellins nei',
'Cross tellin',
'Thin tellin',
'Delicate tellin',
'Timor tellin',
'Virgate tellin',
'Crested hairtail',
'Hilsa shad',
'Toli shad',
'Tigerperches nei',
'Terebellum conch',
'Fly spotted auger',
'Marlinspike',
'Subulate auger',
'Mud creeper',
'Sulcate swamp cerith',
'Auger shells nei',
'Marine turtles nei',
'River and lake turtles nei',
'Spiney-horn octopus',
'Puffers nei',
'Jacona lamprey',
'Chapala lamprey',
'Atlantic white marlin',
'Shortbill spearfish',
'Striped marlin',
'Mediterranean spearfish',
'Roundscale spearfish',
'Longbill spearfish',
'Spearfishes nei',
'Humpback turretfish',
'Atlantic cranch squid',
'Aculeate rock shell',
'Alou rock shell',
'Belligerent rock shell',
'Toad purpura',
'Callao rock shell',
'Chocolate rockshell',
'Crowned rock shell',
'Deltoid rock shell',
'Two-row rock shell',
'Gourd rock shell',
'Nodose rock shell',
'Tuberose rock shell',
'Buller\'s albatross',
'Indian yellow-nosed albatross',
'Shy albatross',
'Atlant. yellow-nosed albatross',
'Grey-headed albatross',
'Chatham Islands albatross',
'Campbell albatross',
'Black-browed albatross',
'Salvin\'s albatross',
'White-capped albatross',
'Antarctic petrel',
'Goldbar wrasse',
'Moon wrasse',
'Surge wrasse',
'Eulachon',
'Prickly redfish',
'Flathead lobster',
'Flathead lobsters nei',
'Alaska pollock(=Walleye poll.)',
'Norwegian pollock',
'Paper thracia',
'Pubescent thracia',
'Baelama anchovy',
'Dussumier\'s thryssa',
'Hamilton\'s thryssa',
'Malabar thryssa',
'Moustached thryssa',
'Longjaw thryssa',
'Orangemouth anchovy',
'Tunas nei',
'Albacore',
'Yellowfin tuna',
'Blackfin tuna',
'Southern bluefin tuna',
'Bigeye tuna',
'Pacific bluefin tuna',
'True tunas nei',
'Atlantic bluefin tuna',
'Longtail tuna',
'Arctic grayling',
'Grayling',
'Southern lobsterette',
'Snoek',
'Black snoek',
'White snake mackerel',
'Bigeye krill',
'Diamondback squid',
'Lybian turrid',
'Shinbone tibia',
'Tench',
'Byron tivela',
'Flat tivela',
'Tivelas nei',
'Pismo clam',
'Triple venus',
'Angolan flying squid',
'Antarctic flying squid',
'Japanese flying squid',
'European flying squid',
'Todarodes flying squids nei',
'Lesser flying squid',
'False gharial',
'Tonga escolar',
'Costate tun',
'Channeled tun',
'Spotted tun',
'Helmet ton',
'Oily tun',
'Pacific partridge tun',
'Banded tun',
'Mosaic tun',
'Tor barb',
'Electric rays nei',
'Black-spotted torpedo',
'Marbled electric ray',
'Electric ray',
'Torpedo rays',
'Common torpedo',
'Totoaba',
'Slimeheads nei',
'Weeverfishes nei',
'Southern pompano',
'Small spotted dart',
'Snubnose pompano',
'Largespotted dart',
'Florida pompano',
'Cayenne pompano',
'Permit',
'Great pompano',
'Longfin pompano',
'Blackblotch pompano',
'Plata pompano',
'Guinean pompano',
'Indian pompano',
'Pompano',
'Paloma pompano',
'Gafftopsail pompano',
'Pompanos nei',
'Steel pompano',
'Shortfin pompano',
'Spotted weever',
'Guinean weever',
'Sailfin weever',
'Greater weever',
'Striped weever',
'Cape Verde weever',
'Starry weever',
'Weevers nei',
'Ribbonfishes',
'Dealfish',
'Mediterranean dealfish',
'Cape horse mackerel',
'Greenback horse mackerel',
'African scad',
'Arabian scad',
'Japanese jack mackerel',
'Rough scad',
'Mediterranean horse mackerel',
'Chilean jack mackerel',
'Blue jack mackerel',
'Jack and horse mackerels nei',
'Pacific jack mackerel',
'Atlantic horse mackerel',
'Cunene horse mackerel',
'Angulate cockle',
'Consors\' cockle',
'Even cockle',
'American yellow cockle',
'Orbit cockle',
'Mexican cockle',
'Slender cockle',
'Giant Pacific cockle',
'Pacific yellow cockle',
'Wrinkled cockle',
'Hardback shrimp',
'Smooth shrimp',
'Carabali shrimp',
'Roughneck shrimp',
'Southern rough shrimp',
'Indio shrimp',
'Brown rough shrimp',
'Pinto shrimp',
'Northern rough shrimp',
'Coarse shrimp',
'Zebra shrimp',
'Malayan rough shrimp',
'Yellow roughneck shrimp',
'Trachypenaeus shrimps nei',
'Spiny scorpionfish',
'Cape scorpionfish',
'Emerald rockcod',
'Scaly rockcod',
'Dusky rockcod',
'Crowned rockcod',
'Trematomus nei',
'Palmate octopus',
'Whitetip reef shark',
'Houndsharks, smoothhounds nei',
'Sharpfin houndshark',
'Spotted houndshark',
'Sharptooth houndshark',
'Banded houndshark',
'Leopard shark',
'Amazonian manatee',
'West Indian manatee',
'West African manatee',
'Hairtails, scabbardfishes nei',
'Pearly hairtail',
'Largehead hairtail',
'Hairtails nei',
'Snakeskin gourami',
'Gouramis nei',
'Crocus giant clam',
'Smooth giant clam',
'Giant clam',
'Elongate giant clam',
'Giant clams nei',
'Fluted giant clam',
'Piper gurnard',
'Gurnards nei',
'Gurnards, searobins nei',
'Threefin blennies nei',
'Half-propellor ark',
'Propellor ark',
'Oval grouper',
'Norway pout',
'Pouting(=Bib)',
'Poor cod',
'Trisopterus nei',
'Cone-shaped top',
'Lined top',
'Maculated top',
'Commercial top',
'Antarctic trophon',
'Comb bittersweet',
'Tessellated bittersweet',
'Silvermouth turban',
'Brown Pacific turban',
'Channelled turban',
'Chestnut turban',
'Goldmouth turban',
'Smooth moon turban',
'Horned turban',
'Coronate moon turban',
'Crass turban',
'Pacific turban',
'Green turban',
'Tapestry turban',
'Stone turban',
'Rough turban',
'Turbans nei',
'Squamate turban',
'Javanese turrid',
'Babylonia turrid',
'Horny auger',
'Duplicate turret',
'Monterosato\'s auger',
'Screw turret',
'Muddy auger',
'Indo-Pacif. bottlenose dolphin',
'Bottlenose dolphin',
'Giant frog shell',
'Agujon needlefish',
'Hound needlefish',
'Needlefishes nei',
'West african fiddler crab',
'Swamp ghost crab',
'Mangrove ghost crab',
'Sea lettuce',
'Chicory sea lettuce',
'Sea lettuces nei',
'Costate top',
'Common button top',
'Canary drum(=Baardman)',
'Argentine croaker',
'Shi drum',
'Drums nei',
'Wakame',
'Wakame nei',
'Megodon oyster',
'Bensasi goatfish',
'Goatfishes',
'Stargazers nei',
'Longspine stargazer',
'West African stargazer',
'Whitespotted stargazer',
'Stargazer',
'Stargazers',
'Whitetongue jack',
'Cottonmouth jack',
'Whitemouth jack',
'Common murre',
'Thick-billed murre',
'Brazilian codling',
'Red hake',
'Southern codling',
'Spotted codling',
'Urophycis nei',
'White hake',
'Bartsch\'s squid',
'Long barrel squid',
'Polar bear',
'Bluespot mullet',
'Valencia toothcarp',
'White-edged lyretail',
'Yellow-edged lyretail',
'Gauntlet vase',
'Spiny vase',
'Caribbean vase',
'Top vase',
'Venus clams nei',
'Golden carpet shell',
'Corrugated venus',
'Durable venus',
'Pullet carpet shell',
'Banded carpet shell',
'Plicate venus',
'Warty venus',
'Asiatic cockle',
'Chinese cockle',
'Vexillate jopas',
'Rugose iter',
'Little-fox miter',
'Singapore scallop',
'Nutmeg melongena',
'Common music volute',
'Green music volute',
'Shuttlecock volva',
'Whale squid',
'Sparkling enope squid',
'Sunburst carrier shell',
'Swordfish',
'Atlantic seabob',
'Pacific seabob',
'Xiphopenaeus shrimps nei',
'Pacific seabobs',
'Pearly razorfish',
'Antarctic yoldia',
'California sea lion',
'Antarctic horsefish',
'Spotted tang',
'Sailfin tang',
'Yellowtail tang',
'Indian zebra sole',
'Dories nei',
'Silvery John dory',
'John dory',
'Angulate volute',
'Cuvier\'s beaked whale',
'Eelpout',
'Eelpouts nei',
'Eel-grass',
'Dwarf eel-grass',
'Grass goby',
'Patagonian scallop',]


# Create the empty folders
for folder_name in folder_names:
  if not os.path.exists(os.path.join(folder_path,folder_name)):
      os.makedirs(os.path.join(folder_path, folder_name))


In [ ]:
count = 0
for folder_name in folder_names:
    if(os.path.isdir(os.path.join(folder_path,folder_name))):
        count = count + 1

print(count)

In [ ]:
ls

In [ ]:
value = 0
dico = {}
for filename in os.listdir('/content/gdrive/MyDrive/HIDE--main/Animal Species'):
        if(os.path.isdir(filename)):
          dico[value]  = filename
          value = value + 1

In [ ]:
 for i in dico.items():
    print(i)

In [ ]:
y = corrupted
print(y)
len(y)

In [ ]:
lis = []
i = 0

while i < len(y):
    print(dico[i])
    count = 0
    for filename in os.listdir('/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]):
        print(filename)
        count  = count + 1
    lis.append(count)
    print("-- nombre de fichiers  = ", lis[i],"--")
    print("\n")
    i = i + 1

In [ ]:
lis

In [ ]:
import hashlib
import os


def hash_directory(path):
    digest = hashlib.sha256()

    for root, dirs, files in os.walk(path):
        for names in files:
            file_path = os.path.join(root, names)

            # Hash the path and add to the digest to account for empty files/directories
            digest.update(hashlib.sha1(file_path[len(path):].encode()).digest())

            # Per @pt12lol - if the goal is uniqueness over repeatability, this is an alternative method using 'hash'
            # digest.update(str(hash(file_path[len(path):])).encode())

            if os.path.isfile(file_path):
                with open(file_path, 'rb') as f_obj:
                    while True:
                        buf = f_obj.read(1024 * 1024)
                        if not buf:
                            break
                        digest.update(buf)

    return digest.hexdigest()

In [ ]:
list_Hash1 = []
for i in range(len(y)):
    list_Hash1.append('')
list_Hash1

In [ ]:
for i in range(len(y)):
    path = '/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]
    #print(path)
    list_Hash1[i] = hash_directory(path)
    print('--hash '+dico[i],'--',list_Hash1[i])

In [ ]:
y

In [ ]:
liste = ['Balistidae', 'Belonidae', 'Enoploteuthidae', 'Cyprinidae', 'Nephropidae', 'Axiidae', 'Serranidae', 'Cardiidae', 'Scombridae', 'Sparidae', 'Acanthuridae', 'Limidae', 'Sergestidae', 'Acipenseridae', 'Acropomatidae', 'Holothuriidae', 'Squalidae', 'Octopodidae', 'Pectinidae', 'Serranidae']

In [ ]:
len(liste)

In [ ]:
i = 0
while i < len(y):
    number = np.random.randint(0,20)
    file = '/'+liste[number]+'.txt'
    if(y[i] == 1):
        my_path = '/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]
        os.chdir(my_path)
        #print(my_path + file)
        if not os.path.exists(my_path + file):
            with open(my_path + file,'w') as fp:
                pass
        else:
            descriptor = open(my_path + file,'a')
            descriptor.write("Species")
            descriptor.close()
        #print('-- folder f'+str(j),' -- file0 -- ',filename)
    i = i + 1

In [ ]:
lis = []
i = 0

while i < len(y):
    print(dico[i])
    count = 0
    for filename in os.listdir('/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]):
        print(filename)
        count  = count + 1
    lis.append(count)
    print("-- nombre de fichiers  = ", lis[i],"--")
    print("\n")
    i = i + 1

In [ ]:
list_Hash2 = []
for i in range(len(y)):
    list_Hash2.append('')
list_Hash2

In [ ]:
for i in range(len(y)):
    path = '/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]
    #print(path)
    list_Hash2[i] = hash_directory(path)
    print('--hash '+dico[i],'--',list_Hash2[i])

In [ ]:
msg_secret = []
for i in range(len(y)):
    if(list_Hash1[i] != list_Hash2[i]):
        msg_secret.append(1)
    else:
        msg_secret.append(0)

In [ ]:
msg_secret

In [ ]:
len(msg_secret)

In [ ]:
np.array(msg_secret, dtype = 'i')

In [ ]:
decoded = qc_ldpc.decode(np.copy(np.array(msg_secret, dtype = 'i')))
decoded = qc_ldpc.get_message(decoded)
print(decoded)
print(len(decoded))

In [ ]:
word

In [ ]:
try:
    assert (msg_secret == y).all()
    assert (decoded == word).all()
except AssertionError:
    print("The secret message must be forwarded")

# #Code script for time evaluation

In [7]:
import time
from google.colab import drive
import os
import shutil

drive.mount('/content/gdrive')

os.chdir('/content/gdrive/MyDrive/HIDE--main')

import numpy as np
from utils import gaussjordan
from LinearCode import LinearCode

start = time.time()

class QC_LDPC(LinearCode):
    """
    Quasi-Cyclic LDPC code representation (extends LinearCode)

    ...

    Methods
    -------
    from_params(n, p, w)
        Init QC-LDPC by length, circulant size and code weight

    _get_circulant_block(polynom)
        Get circulant (p, p) for given vector of size p

    """

    def __init__(self, G, H):
        super().__init__(G, H)

    @classmethod
    def from_params(cls, n, p, w):
        assert n % p == 0, "p must be delimeter of n"

        n0 = n // p
        assert w > 2*n0, "not enough code weight"

        fine = False

        while not fine:
            blocks = []
            inverse_block = None
            inverse_block_position = None

            vector = [1 for _ in range(w)] + [0 for _  in range(n - w)]
            vector = np.array(vector, dtype=int)
            np.random.shuffle(vector)

            for i in range(n0):
                circ = vector[i*p:(i+1)*p]

                if sum(circ) < 2:
                    inverse_block = None
                    break

                block = QC_LDPC._get_circulant_block(circ)
                blocks.append(block)

                A, P = gaussjordan(block, True)
                A = np.array(A, dtype=int)
                P = np.array(P, dtype=int)

                if (A == np.eye(p, dtype=int)).all():
                    inverse_block_position = i
                    inverse_block = P

            # continue only if inverse circulant found
            fine = True if inverse_block is not None else False

        # put inverse block on last position
        blocks[inverse_block_position], blocks[n0-1] = blocks[n0-1], blocks[inverse_block_position]
        H = np.concatenate(blocks, axis=1)

        for i in range(n0):
            blocks[i] = blocks[i] @ inverse_block % 2
            blocks[i] = blocks[i].T

        Ht = np.concatenate(blocks[:n0-1], axis=0)
        G = np.concatenate((np.eye(Ht.shape[0], dtype=int), Ht), axis=1)

        assert (G @ H.T % 2 == 0).all(), "G is not correspond to H"

        return cls(G, H)

    @staticmethod
    def _get_circulant_block(polynom):
        N = len(polynom)
        block = np.empty((N, N), dtype=int)

        for i in range(N):
            block[i] = np.roll(polynom, i)

        return block



# EXAMPLE USAGE:

n = 16
p = 8
w = 7

qc_ldpc = QC_LDPC.from_params(n, p, w)
print(qc_ldpc.getG())
print(qc_ldpc.getG().shape)

print(qc_ldpc.getH())
print(qc_ldpc.getH().shape)

word = np.random.randint(2, size=qc_ldpc.getG().shape[0])



word

print(len(word))

encoded = qc_ldpc.encode(word)
print(encoded)
print(len(encoded))

# error vector size n with t or less errors
errors_num = 2
e = [1 for _ in range(errors_num)] + [0 for _  in range(n - errors_num)]
e = np.array(e)
np.random.shuffle(e)
print(e)
print(len(e))

corrupted = (encoded + e) % 2
print(corrupted)
print(len(corrupted))

os.chdir('/content/gdrive/MyDrive/HIDE--main/Animal Species/')

#print(ls)

value = 0
dico = {}
for filename in os.listdir('/content/gdrive/MyDrive/HIDE--main/Animal Species/'):
        if(os.path.isdir(filename)):
            dico[value]  = filename
            value = value + 1


for i in dico.items():
    print(i)

y = corrupted
print(y)
len(y)

i = 0
while i < len(y):
    my_path = '/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]
    os.chdir(my_path)
    if os.path.isfile(my_path + '/file.txt'):
        os.remove(my_path + '/file.txt')
        #print('-- folder f'+str(j),' -- file0 -- ',filename)
    i = i + 1

lis = []
i = 0

while i < len(y):
    print(dico[i])
    count = 0
    for filename in os.listdir('/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]):
        print(filename)
        count  = count + 1
    lis.append(count)
    print("-- nombre de fichiers  = ", lis[i],"--")
    print("\n")
    i = i + 1

print(lis)

import hashlib
import os


def hash_directory(path):
    digest = hashlib.sha256()

    for root, dirs, files in os.walk(path):
        for names in files:
            file_path = os.path.join(root, names)

            # Hash the path and add to the digest to account for empty files/directories
            digest.update(hashlib.sha1(file_path[len(path):].encode()).digest())

            # Per @pt12lol - if the goal is uniqueness over repeatability, this is an alternative method using 'hash'
            # digest.update(str(hash(file_path[len(path):])).encode())

            if os.path.isfile(file_path):
                with open(file_path, 'rb') as f_obj:
                    while True:
                        buf = f_obj.read(1024 * 1024)
                        if not buf:
                            break
                        digest.update(buf)

    return digest.hexdigest()

list_Hash1 = []
for i in range(len(y)):
    list_Hash1.append('')
list_Hash1

for i in range(len(y)):
    path = '/content/gdrive/My Drive/Animal Species/' + dico[i]
    #print(path)
    list_Hash1[i] = hash_directory(path)
    print('--hash '+dico[i],'--',list_Hash1[i])

print(y)

liste = ['Balistidae', 'Belonidae', 'Enoploteuthidae', 'Cyprinidae', 'Nephropidae', 'Axiidae', 'Serranidae', 'Cardiidae', 'Scombridae', 'Sparidae', 'Acanthuridae', 'Limidae', 'Sergestidae', 'Acipenseridae', 'Acropomatidae', 'Holothuriidae', 'Squalidae', 'Octopodidae', 'Pectinidae', 'Serranidae']

i = 0
while i < len(y):
    number = np.random.randint(0,20)
    file = '/'+liste[number]+'.txt'
    if(y[i] == 1):
        my_path = '/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]
        os.chdir(my_path)
        #print(my_path + file)
        if not os.path.exists(my_path + file):
            with open(my_path + file,'w') as fp:
                pass
        else:
            descriptor = open(my_path + file,'a')
            descriptor.write("Species")
            descriptor.close()
        #print('-- folder f'+str(j),' -- file0 -- ',filename)
    i = i + 1

lis = []
i = 0

while i < len(y):
    print(dico[i])
    count = 0
    for filename in os.listdir('/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]):
        print(filename)
        count  = count + 1
    lis.append(count)
    print("-- nombre de fichiers  = ", lis[i],"--")
    print("\n")
    i = i + 1



list_Hash2 = []
for i in range(len(y)):
    list_Hash2.append('')
list_Hash2

for i in range(len(y)):
    path = '/content/gdrive/MyDrive/HIDE--main/Animal Species/' + dico[i]
    #print(path)
    list_Hash2[i] = hash_directory(path)
    print('--hash '+dico[i],'--',list_Hash2[i])

msg_secret = []
for i in range(len(y)):
    if(list_Hash1[i] != list_Hash2[i]):
        msg_secret.append(1)
    else:
        msg_secret.append(0)

print(np.array(msg_secret, dtype = 'i'))


decoded = qc_ldpc.decode(np.copy(np.array(msg_secret, dtype = 'i')))
decoded = qc_ldpc.get_message(decoded)
print(decoded)
print(len(decoded))

print(word)

try:
    assert (msg_secret == y).all()
    assert (decoded == word).all()
except AssertionError:
    print("The secret message must be forwarded")



end = time.time()
print('time = ',end - start)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
(396, 'Parrotfish')
(397, 'Redfin parrotfish')
(398, 'Gilthead seabream')
(399, 'Violet sea urchin')
(400, 'False gorgon')
(401, 'Penguins nei')
(402, 'Northern puffer')
(403, 'Bandtail puffer')
(404, 'Atlantic puffers nei')
(405, 'Guinean barracuda')
(406, 'Great barracuda')
(407, 'Mexican barracuda')
(408, 'Bigeye barracuda')
(409, 'Pickhandle barracuda')
(410, 'Obtuse barracuda')
(411, 'European barracuda')
(412, 'Barracudas nei')
(413, 'Yellowmouth barracuda')
(414, 'Barracudas, etc. nei')
(415, 'Scalloped bonnethead')
(416, 'Whitefin hammerhead')
(417, 'Scalloped hammerhead')
(418, 'Scoophead')
(419, 'Great hammerhead')
(420, 'Hammerhead sharks nei')
(421, 'Bonnethead')
(422, 'Smalleye hammerhead')
(423, 'Smooth hammerhead')
(424, 'Hammerhead sharks, etc. nei')
(425, 'Bigeye picarel')
(426, 'Blotched picarel')
(427, 'Blackspot picarel')
(428, 'Blacktail picarel')
(429, 'Picarel')
(430, 'Picarels nei')
(4